# 1차 데이터 정제

기본적인 데이터 중요도를 판단하기 위해 기초적인 정제를 진행합니다.

데이터의 맥락보다는 데이터 자체를 살리고 모든 데이터에 대해 숫자형으로 인코딩을 진행합니다.

In [1]:
import pandas as pd

In [2]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df_main = pd.read_csv('data/transactions_data.csv')

df_main = (
    df_main
    .drop('date', axis=1)
    .drop('use_chip', axis=1)
    .drop('merchant_id', axis=1)
    .drop('merchant_city', axis=1)
    .drop('zip', axis=1)
    .drop('mcc', axis=1)
)

df_main = df_main[df_main['errors'].isnull()]
df_main['merchant_state'] = df_main['merchant_state'].fillna('Online')
df_main['merchant_state'] = le.fit_transform(df_main['merchant_state'])
df_main['amount'] = df_main['amount'].str.replace('$', '', regex=False).str.replace(',', '').astype(float)
df_main = df_main.drop('errors', axis=1)

df_main.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13094522 entries, 0 to 13305914
Data columns (total 5 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   client_id       int64  
 2   card_id         int64  
 3   amount          float64
 4   merchant_state  int64  
dtypes: float64(1), int64(4)
memory usage: 599.4 MB


In [3]:
df_fraud = pd.read_csv('data/train_fraud_labels.csv')

df_fraud['target'] = df_fraud['target'].map({'Yes':1, 'No':0}).astype(int)
df_fraud = df_fraud.rename(columns={'target':'is_fraud'})
df_fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914963 entries, 0 to 8914962
Data columns (total 2 columns):
 #   Column    Dtype
---  ------    -----
 0   id        int64
 1   is_fraud  int64
dtypes: int64(2)
memory usage: 136.0 MB


In [4]:
df_users = pd.read_csv('data/users_data.csv')

df_users = (
    df_users
    .drop('retirement_age', axis=1)
    .drop('birth_year', axis=1)
    .drop('birth_month', axis=1)
    .drop('address', axis=1)
    .drop('latitude', axis=1)
    .drop('longitude', axis=1)
    .drop('per_capita_income', axis=1)
    .drop('credit_score', axis=1)
    .drop('num_credit_cards', axis=1)
)
df_users['yearly_income'] = df_users['yearly_income'].str.replace('$', '', regex=False).str.replace(',', '').astype(float)
df_users['total_debt'] = df_users['total_debt'].str.replace('$', '', regex=False).str.replace(',', '').astype(float)
df_users['debt/salary_ratio'] = round(df_users['total_debt']/df_users['yearly_income']*100,2)
df_users = df_users.drop('yearly_income', axis=1).drop('total_debt', axis=1)
df_users['gender'] = df_users['gender'].map({'Male':1, 'Female':0}).astype(int)
df_users = df_users.rename(columns={'id':'client_id', 'gender':'is_male'})

df_users.info()
print()
print(df_users.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   client_id          2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   is_male            2000 non-null   int64  
 3   debt/salary_ratio  2000 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 62.6 KB

   client_id  current_age  is_male  debt/salary_ratio
0        825           53        0             213.77
1       1746           53        0             247.69
2       1718           81        0               0.59
3        708           63        0              80.96
4       1164           43        1             167.62


In [5]:
from sklearn.preprocessing import OrdinalEncoder

df_cards = pd.read_csv('data/cards_data.csv')
brands = [['Visa', 'Mastercard', 'Discover', 'Amex']]
types = [['Credit', 'Debit', 'Debit (Prepaid)']]
ode_brand = OrdinalEncoder(categories=brands)
ode_type = OrdinalEncoder(categories=types)

df_cards = (
    df_cards
    .drop('client_id', axis=1)
    .drop('card_number', axis=1)
    .drop('expires', axis=1)
    .drop('cvv', axis=1)
    .drop('has_chip', axis=1)
    .drop('num_cards_issued', axis=1)
    .drop('credit_limit', axis=1)
    .drop('acct_open_date', axis=1)
    .drop('card_on_dark_web', axis=1)
)
df_cards['card_brand'] = ode_brand.fit_transform(df_cards[['card_brand']])
df_cards['card_type'] = ode_type.fit_transform(df_cards[['card_type']])
df_cards['card_brand'] = df_cards['card_brand'].astype(int)
df_cards['card_type'] = df_cards['card_type'].astype(int)
df_cards = df_cards.rename(columns={'id':'card_id'})

df_cards.info()
print()
print(df_cards.head(15))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   card_id                6146 non-null   int64
 1   card_brand             6146 non-null   int64
 2   card_type              6146 non-null   int64
 3   year_pin_last_changed  6146 non-null   int64
dtypes: int64(4)
memory usage: 192.2 KB

    card_id  card_brand  card_type  year_pin_last_changed
0      4524           0          1                   2008
1      2731           0          1                   2014
2      3701           0          1                   2004
3        42           0          0                   2012
4      4659           1          2                   2009
5      4537           0          0                   2012
6      1278           0          1                   2011
7      3687           1          1                   2015
8      3465           1        

In [8]:
df_merged = pd.merge(df_main, df_users, on='client_id', how='inner')
df_merged = pd.merge(df_merged, df_cards, on='card_id', how='inner')
df_merged = pd.merge(df_merged, df_fraud, on='id', how='inner')
df_merged = (
    df_merged
    .drop('id', axis=1)
    .drop('client_id', axis=1)
    .drop('card_id', axis=1)
)

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8773196 entries, 0 to 8773195
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   amount                 float64
 1   merchant_state         int64  
 2   current_age            int64  
 3   is_male                int64  
 4   debt/salary_ratio      float64
 5   card_brand             int64  
 6   card_type              int64  
 7   year_pin_last_changed  int64  
 8   is_fraud               int64  
dtypes: float64(2), int64(7)
memory usage: 602.4 MB


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, average_precision_score

# 1. 데이터 전처리 (제안해주신 df_merged가 있다고 가정)
# -----------------------------------------------------------
# (1) Amount: 절댓값 처리
df_merged['amount'] = df_merged['amount'].abs()

# (2) Year PIN: 경과 연수로 변환 (현재 2026년 기준 가정, 데이터셋 시점에 맞게 조정 필요)
# 데이터셋이 과거 데이터라면 해당 데이터의 최대 연도를 기준으로 빼야 합니다.
max_year = df_merged['year_pin_last_changed'].max()
df_merged['pin_years_gap'] = max_year - df_merged['year_pin_last_changed']
df_merged = df_merged.drop('year_pin_last_changed', axis=1)

# (3) Merchant State: 빈도 인코딩 (Frequency Encoding)
# 모델이 모르는 지역이 들어와도 '빈도 0' 혹은 '평균 빈도'로 처리 가능해짐
state_counts = df_merged['merchant_state'].value_counts(normalize=True)
df_merged['merchant_state_freq'] = df_merged['merchant_state'].map(state_counts)
# 원본 라벨 컬럼은 유지하거나 삭제 (트리 모델은 둘 다 있어도 잘 처리함)
# df_merged = df_merged.drop('merchant_state', axis=1) 

# 2. 데이터 분할 (Stratified Split 필수!)
# -----------------------------------------------------------
X = df_merged.drop('is_fraud', axis=1)
y = df_merged['is_fraud']

# Train(60%), Temp(40%) 분리
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=42, stratify=y
)

# Temp를 다시 Validation(20%), Test(20%)로 분리
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Train shape: {X_train.shape}, Val shape: {X_val.shape}, Test shape: {X_test.shape}")
print(f"Fraud ratio in Train: {y_train.mean():.4f}") # 약 0.0015 확인 필요


Train shape: (5263917, 9), Val shape: (1754639, 9), Test shape: (1754640, 9)
Fraud ratio in Train: 0.0015


# 학습 1

1차 정제 데이터를 활용하여 XGBoost를 통한 모델학습을 진행합니다.

파라미터는 scale_pos_weight = 실제 (정상 데이터 수 / 사기 데이터 수) 배율을 그대로 사용합니다. (약 686)

결과 : precision = 0.01, recall = 0.90

precision이 너무 낮습니다. 즉 너무 많은 데이터(거의 전체의 10%비율)를 fraud로 판단하였습니다.

In [18]:
# 3. 모델 학습 (XGBoost)
# -----------------------------------------------------------
# 불균형 데이터 가중치 계산: (정상 데이터 수) / (사기 데이터 수)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Calculated scale_pos_weight: {scale_pos_weight:.2f}")

model = XGBClassifier(
    n_estimators=100,      # 트리 개수 (적절히 조절)
    learning_rate=0.1,
    max_depth=6,           # 트리 깊이
    scale_pos_weight=scale_pos_weight, # [핵심] 불균형 데이터 처리를 위한 가중치
    eval_metric='auc',   # 평가 지표: AUC
    n_jobs=-1,             # 모든 CPU 코어 사용
    random_state=42
)

print("🚀 학습 시작 (데이터가 많아 시간이 좀 걸릴 수 있습니다)...")
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=10
)

# 4. 성능 평가
# -----------------------------------------------------------
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1] # 확률값

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))

print(f"\n=== AUPRC Score: {average_precision_score(y_test, y_prob):.4f} ===")

Calculated scale_pos_weight: 686.37
🚀 학습 시작 (데이터가 많아 시간이 좀 걸릴 수 있습니다)...
[0]	validation_0-auc:0.94859
[10]	validation_0-auc:0.94986
[20]	validation_0-auc:0.95168
[30]	validation_0-auc:0.95277
[40]	validation_0-auc:0.95460
[50]	validation_0-auc:0.95508
[60]	validation_0-auc:0.95595
[70]	validation_0-auc:0.95721
[80]	validation_0-auc:0.95809
[90]	validation_0-auc:0.95879
[99]	validation_0-auc:0.95906

=== Confusion Matrix ===
[[1595317  156770]
 [    246    2307]]

=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      0.91      0.95   1752087
           1       0.01      0.90      0.03      2553

    accuracy                           0.91   1754640
   macro avg       0.51      0.91      0.49   1754640
weighted avg       1.00      0.91      0.95   1754640


=== AUPRC Score: 0.3144 ===


# 학습 2

1차 학습에서 파라미터를 scale_pos_weight = 25로 기존의 제곱근 이하의 수치로 변경합니다.

또한 threshold = 0.8를 적용하여 precision을 낮추는 것을 유도합니다.

결과 : precision = 0.67, recall = 0.30

precision이 급격하게 상승했습니다. 하지만 사기로 판단하는 건수 자체는 비약적으로 줄어들어 70%의 사기를 놓쳤습니다.

In [19]:
# 3. 모델 학습 (XGBoost)
# -----------------------------------------------------------
# 불균형 데이터 가중치 계산: (정상 데이터 수) / (사기 데이터 수)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Calculated scale_pos_weight: {scale_pos_weight:.2f}")

model = XGBClassifier(
    n_estimators=100,      # 트리 개수 (적절히 조절)
    learning_rate=0.1,
    max_depth=6,           # 트리 깊이
    scale_pos_weight=25, # [핵심] 불균형 데이터 처리를 위한 가중치
    eval_metric='auc',   # 평가 지표: Area Under Precision-Recall Curve (불균형에 좋음)
    n_jobs=-1,             # 모든 CPU 코어 사용
    random_state=42
)

print("🚀 학습 시작 (데이터가 많아 시간이 좀 걸릴 수 있습니다)...")
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=10
)

# 4. 성능 평가
# -----------------------------------------------------------
y_prob = model.predict_proba(X_test)[:, 1] # 확률값
threshold = 0.8  # 0.5 대신 threshold 이상일 때만 사기로 판단
y_pred = (y_prob >= threshold).astype(int)

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))

print(f"\n=== AUPRC Score: {average_precision_score(y_test, y_prob):.4f} ===")

Calculated scale_pos_weight: 686.37
🚀 학습 시작 (데이터가 많아 시간이 좀 걸릴 수 있습니다)...
[0]	validation_0-auc:0.94753
[10]	validation_0-auc:0.95002
[20]	validation_0-auc:0.95117
[30]	validation_0-auc:0.95198
[40]	validation_0-auc:0.95345
[50]	validation_0-auc:0.95551
[60]	validation_0-auc:0.95744
[70]	validation_0-auc:0.95856
[80]	validation_0-auc:0.95911
[90]	validation_0-auc:0.95980
[99]	validation_0-auc:0.96058

=== Confusion Matrix ===
[[1751701     386]
 [   1786     767]]

=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1752087
           1       0.67      0.30      0.41      2553

    accuracy                           1.00   1754640
   macro avg       0.83      0.65      0.71   1754640
weighted avg       1.00      1.00      1.00   1754640


=== AUPRC Score: 0.3426 ===


# 학습 3

데이터 비율조절의 필요성을 판단하기 위해 scale_pos_weight 파라미터를 아예 제거합니다.

threshold = 0.9를 적용하여 precision을 더더욱 낮추는 것을 유도합니다.

결과 : precision = 0.98, recall = 0.05

recall이 너무 낮은 것으로 보아 모델이 사기에 대한 패턴을 제대로 학습하지 못했습니다.

정상-사기 데이터 비율 조정은 반드시 필요한 것으로 판단됩니다.

In [20]:
# 3. 모델 학습 (XGBoost)
# -----------------------------------------------------------
# 불균형 데이터 가중치 계산: (정상 데이터 수) / (사기 데이터 수)
scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
print(f"Calculated scale_pos_weight: {scale_pos_weight:.2f}")

model = XGBClassifier(
    n_estimators=100,      # 트리 개수 (적절히 조절)
    learning_rate=0.1,
    max_depth=6,           # 트리 깊이
    eval_metric='auc',   # 평가 지표: Area Under Precision-Recall Curve (불균형에 좋음)
    n_jobs=-1,             # 모든 CPU 코어 사용
    random_state=42
)

print("🚀 학습 시작 (데이터가 많아 시간이 좀 걸릴 수 있습니다)...")
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=10
)

# 4. 성능 평가
# -----------------------------------------------------------
y_prob = model.predict_proba(X_test)[:, 1] # 확률값
threshold = 0.9  # 0.5 대신 threshold 이상일 때만 사기로 판단
y_pred = (y_prob >= threshold).astype(int)

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))

print(f"\n=== AUPRC Score: {average_precision_score(y_test, y_prob):.4f} ===")

Calculated scale_pos_weight: 686.37
🚀 학습 시작 (데이터가 많아 시간이 좀 걸릴 수 있습니다)...
[0]	validation_0-auc:0.94698
[10]	validation_0-auc:0.94732
[20]	validation_0-auc:0.94901
[30]	validation_0-auc:0.95103
[40]	validation_0-auc:0.95277
[50]	validation_0-auc:0.95393
[60]	validation_0-auc:0.95510
[70]	validation_0-auc:0.95573
[80]	validation_0-auc:0.95663
[90]	validation_0-auc:0.95704
[99]	validation_0-auc:0.95743

=== Confusion Matrix ===
[[1752085       2]
 [   2431     122]]

=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1752087
           1       0.98      0.05      0.09      2553

    accuracy                           1.00   1754640
   macro avg       0.99      0.52      0.55   1754640
weighted avg       1.00      1.00      1.00   1754640


=== AUPRC Score: 0.3573 ===


# 학습 4

앙상블을 통한 XGBoost + RandomForest 동시학습을 진행합니다.

정상 데이터의 under_sampling을 통해 정상:사기 데이터 비율을 10:1로 줄입니다.

목표 precision = 0.999에 도달하는 threshold값을 탐색합니다.

결과 : 

threshold = 0.995, precision = 0.7721, recall = 0.1128


threshold = 0.999, precision = 1.0000, recall = 0.0027

모델이 fraud를 판단할 근거 자체가 부족한 상태라는 결론입니다.

데이터 정제를 다시 진행하되, 특히 거래의 맥락 파악 부분에 신경을 써서 다시 만들 필요가 있습니다.

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler

# 1. 데이터 전처리 (이전과 동일)
# -----------------------------------------------------------
if 'amount' in df_merged.columns:
    df_merged['amount'] = df_merged['amount'].abs()

if 'year_pin_last_changed' in df_merged.columns:
    max_year = df_merged['year_pin_last_changed'].max()
    df_merged['pin_years_gap'] = max_year - df_merged['year_pin_last_changed']
    df_merged = df_merged.drop('year_pin_last_changed', axis=1)

# Merchant State 빈도 인코딩 (필수)
if 'merchant_state' in df_merged.columns and df_merged['merchant_state'].dtype == 'int64':
    state_counts = df_merged['merchant_state'].value_counts(normalize=True)
    df_merged['merchant_state_freq'] = df_merged['merchant_state'].map(state_counts)
    # df_merged = df_merged.drop('merchant_state', axis=1) # 선택 사항

X = df_merged.drop('is_fraud', axis=1)
y = df_merged['is_fraud']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. 보수적인 샘플링 (Undersampling Only)
# -----------------------------------------------------------
# 정상:사기 비율을 10:1 (0.1) 정도로 맞춥니다.
# 정상을 너무 많이 버리면 "정상 패턴"을 학습 못해서 오탐(FP)이 늘어납니다.
# 10:1 정도면 모델이 "아, 정상이 훨씬 많구나"라고 인지하여 보수적으로 변합니다.
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(f"Resampled Train shape: {X_resampled.shape}")
print(f"Resampled Fraud ratio: {y_resampled.mean():.4f}") # 약 0.09 (9%)

# 3. 모델 정의 (가중치 제거 & 보수적 설정)
# -----------------------------------------------------------
# (1) XGBoost: scale_pos_weight 제거 (기본값 1)
xgb_clf = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,        # 깊이를 줄여서 과적합 방지
    scale_pos_weight=1, # [핵심] 가중치를 주지 않음 -> Precision 중시
    eval_metric='auc',
    n_jobs=-1,
    random_state=42
)

# (2) Random Forest: Precision 확보에 유리
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=12,       # 너무 깊지 않게
    n_jobs=-1,
    random_state=42
)

# 4. 앙상블 학습
# -----------------------------------------------------------
voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('rf', rf_clf)],
    voting='soft'
)

print("🚀 스나이퍼 모델 학습 시작...")
voting_clf.fit(X_resampled, y_resampled)

# 5. 최적의 Threshold 탐색
# -----------------------------------------------------------
# 검증 데이터에 대한 확률 예측
y_prob = voting_clf.predict_proba(X_test)[:, 1]

print("\n=== 🎯 Threshold Tuning for Precision >= 0.99 ===")
print(f"{'Threshold':<10} | {'Precision':<10} | {'Recall':<10} | {'FP Count':<10} | {'TP Count':<10}")
print("-" * 60)

best_th = 0.5
found = False

# 0.5부터 0.999까지 촘촘하게 검사
thresholds = np.arange(0.5, 1.0, 0.05).tolist() + [0.95, 0.96, 0.97, 0.98, 0.99, 0.995, 0.999]
thresholds = sorted(list(set(thresholds)))

for th in thresholds:
    y_pred_th = (y_prob >= th).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_th).ravel()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    print(f"{th:<10.3f} | {precision:<10.4f} | {recall:<10.4f} | {fp:<10} | {tp:<10}")
    
    # 목표 달성 조건 확인 (Precision >= 0.99 이면서 Recall이 0이 아닌 경우)
    if precision >= 0.99 and recall > 0.0:
        if not found: # 조건을 만족하는 첫 번째(가장 낮은) Threshold 저장
            best_th = th
            found = True
        # 더 높은 Recall을 찾기 위해 best_th를 계속 갱신할 수도 있음 (여기서는 첫 달성 지점 기준)

print("-" * 60)

if found:
    print(f"\n✅ 목표 달성! 추천 Threshold: {best_th}")
    
    # 최종 결과 출력
    final_pred = (y_prob >= best_th).astype(int)
    print(f"\n=== Final Report (Threshold: {best_th}) ===")
    print(confusion_matrix(y_test, final_pred))
    print(classification_report(y_test, final_pred))
else:
    print("\n⚠️ Precision 0.99를 달성하지 못했습니다. 더 강력한 Feature가 필요합니다.")

Resampled Train shape: (112310, 9)
Resampled Fraud ratio: 0.0909
🚀 스나이퍼 모델 학습 시작...

=== 🎯 Threshold Tuning for Precision >= 0.99 ===
Threshold  | Precision  | Recall     | FP Count   | TP Count  
------------------------------------------------------------
0.500      | 0.0503     | 0.7047     | 33959      | 1799      
0.550      | 0.0636     | 0.6479     | 24356      | 1654      
0.600      | 0.0878     | 0.5734     | 15209      | 1464      
0.650      | 0.1262     | 0.4990     | 8825       | 1274      
0.700      | 0.1912     | 0.4418     | 4772       | 1128      
0.750      | 0.3080     | 0.3897     | 2236       | 995       
0.800      | 0.4837     | 0.3486     | 950        | 890       
0.850      | 0.6188     | 0.3173     | 499        | 810       
0.900      | 0.6715     | 0.3059     | 382        | 781       
0.950      | 0.7171     | 0.2949     | 297        | 753       
0.950      | 0.7171     | 0.2949     | 297        | 753       
0.960      | 0.7228     | 0.2859     | 280       

# 2차 데이터 정제

기존에 버려졌던 데이터를 되살리고 모델학습에 유효한 숫자형 데이터로 정제합니다.

1. 핵심 수치

'amount', # 거래 금액(절대값)

'utilization_ratio', # 한도 대비 금액

'amount_income_ratio', # 소득 대비 금액

2. 기술적/패턴 맥락

'tech_mismatch', # IC카드인데 마그네틱 거래를 진행했는지 여부

'pin_years_gap', # 비밀번호 변경 오래됐는지

'num_credit_cards', # 카드 개수 (다중 발급 사기 연관)

3. 시간 맥락

'hour', # 거래 시간(시)

'is_night', # 거래가 00시~06시 사이에 이루어졌는지 여부

4. 위치/업종 맥락 (Risk Score로 변환됨)

'mcc_risk', # 업종 위험도

'state_risk', # 주(State) 위험도

'zip_risk', # 지역(Zip) 위험도

5. 사용자 정보 (보조)

'current_age', # 카드 명의자 나이

'credit_score', # 신용점수 낮으면 위험할 수 있음

6. 타겟 (정답)

'is_fraud'

In [1]:
import pandas as pd
import numpy as np

# 1. 데이터 로드 (파일 경로는 실제 환경에 맞춰주세요)
# transactions_data는 워낙 크니 chunk로 나누거나 dask를 써야 할 수도 있지만, 
# 여기선 pandas로 로드 가능하다고 가정하고 진행합니다.
print("⏳ Loading Data...")

# (1) Users & Cards
users = pd.read_csv('data/users_data.csv')
cards = pd.read_csv('data/cards_data.csv')
users = users.rename(columns={'id':'client_id'})
cards = cards.rename(columns={'id':'card_id'})

# (2) Transactions (예시: 필요한 컬럼만 불러와서 메모리 절약 가능)
# use_chip, merchant_city, zip, mcc 등 버리지 말고 다 가져옵니다.
trans = pd.read_csv('data/transactions_data.csv') 

# (3) Labels (Target)
labels = pd.read_csv('data/train_fraud_labels.csv') # id, target('Yes'/'No')

# 2. 데이터 병합 (Merge)
print("⏳ Merging Data...")

# Transactions + Labels (id 기준)
# 타겟이 없는 데이터는 학습에 쓸 수 없으므로 Inner Join
df = pd.merge(trans, labels, on='id', how='inner')

# + Users (client_id 기준)
df = pd.merge(df, users, on='client_id', how='left')

# + Cards (card_id 기준)
# client_id도 같이 키로 잡으면 더 안전합니다.
df = pd.merge(df, cards, on=['card_id', 'client_id'], how='left')

print(f"✅ Merged Shape: {df.shape}")

⏳ Loading Data...
⏳ Merging Data...
✅ Merged Shape: (8914963, 37)


In [2]:
# 3. 기초 정제 함수
def clean_dollar(x):
    if isinstance(x, str):
        return float(x.replace('$', '').replace(',', ''))
    return x

print("⏳ Cleaning Data...")

# (1) 금액 관련 컬럼 $ 제거 및 실수형 변환
money_cols = ['amount', 'per_capita_income', 'yearly_income', 'total_debt', 'credit_limit']
for col in money_cols:
    if col in df.columns:
        df[col] = df[col].apply(clean_dollar)

# (2) Amount 절대값 처리 (환불 데이터도 규모로 판단)
df['amount'] = df['amount'].abs()

# (3) Target 변환 (Yes/No -> 1/0)
df['is_fraud'] = df['target'].apply(lambda x: 1 if x == 'Yes' else 0)
df = df.drop(columns=['target']) # 기존 문자열 컬럼 제거

# (4) Online Transaction 처리
# merchant_state가 비어있으면(NaN) -> 'Online'으로 채움 (작성자님 전략 유지)
df['merchant_state'] = df['merchant_state'].fillna('Online')

# (5) 카드 정보 결측치 처리 (혹시 Join 안 된 경우)
df['has_chip'] = df['has_chip'].fillna('NO') # 보수적으로 없음 처리

print("✅ Cleaning Complete")

⏳ Cleaning Data...
✅ Cleaning Complete


In [3]:
print("⏳ Engineering Sniper Features...")

# [Feature 1] 기술적 모순 (Tech Mismatch)
# 칩이 있는 카드(YES)인데, 결제는 긁어서(Swipe) 했다? -> 복제카드 의심
# use_chip: 'Swipe Transaction', 'Chip Transaction', 'Online Transaction'
# has_chip: 'YES', 'NO'
def check_tech_mismatch(row):
    # 온라인 거래는 칩 여부와 상관 없음
    if row['use_chip'] == 'Online Transaction':
        return 0
    # 칩이 있는데 스와이프 했으면 의심 (1)
    if row['has_chip'] == 'YES' and row['use_chip'] == 'Swipe Transaction':
        return 1
    return 0

df['tech_mismatch'] = df.apply(check_tech_mismatch, axis=1)

# [Feature 2] 한도 소진율 (Utilization Ratio)
# 500달러 결제라도, 한도가 500달러인 카드를 꽉 채워 쓴 거면 위험함 (속칭 '카드깡' 등)
df['utilization_ratio'] = df['amount'] / df['credit_limit']

# [Feature 3] 소득 대비 지출 비율 (Income Ratio)
# 연봉 3만불인 사람이 한 번에 1만불을 긁으면 이상함
# yearly_income이 0일 수도 있으니 안전하게 처리
df['amount_income_ratio'] = df['amount'] / (df['yearly_income'] + 1) 

# [Feature 4] 시간대 정보 (Hour)
# 문자열 date를 datetime으로 변환
df['date_obj'] = pd.to_datetime(df['date'])
df['hour'] = df['date_obj'].dt.hour
df['is_night'] = df['hour'].apply(lambda x: 1 if 0 <= x < 6 else 0) # 새벽 0시~6시

# [Feature 5] PIN 변경 경과 기간
# 최근에 PIN을 바꿨는데(해킹 등) 고액 결제? 
# 현재 시점을 데이터의 최대 연도로 가정 (2020년)
current_year = df['date_obj'].dt.year.max()
df['pin_years_gap'] = current_year - df['year_pin_last_changed']

# [Feature 6] MCC Risk Mapping (Target Encoding)
# MCC 코드 자체가 중요한 게 아니라, "그 업종에서 사기가 얼마나 빈번한가"가 중요함.
# Train 데이터 기준으로 각 MCC별 사기 확률을 계산해서 매핑합니다.
# 주의: Data Leakage 방지를 위해 원칙적으로는 Train Split 후에 해야 하지만, 
# 데이터가 워낙 크고 분포가 일정하다면 전체 기준으로 맵을 만들어도 무방합니다.
mcc_risk_map = df.groupby('mcc')['is_fraud'].mean()
df['mcc_risk'] = df['mcc'].map(mcc_risk_map)

# [Feature 7] State Risk Mapping
# "CA는 199번"이라는 숫자보다 "CA는 사기율 0.2%"라는 숫자가 모델에겐 훨씬 유익함.
state_risk_map = df.groupby('merchant_state')['is_fraud'].mean()
df['state_risk'] = df['merchant_state'].map(state_risk_map)

# [Feature 8] Zip code 활용 (Distance 대체제)
# Zip code는 너무 많으므로, Zip code별 사기 확률(Risk)로 변환
zip_risk_map = df.groupby('zip')['is_fraud'].mean()
df['zip_risk'] = df['zip'].map(zip_risk_map)
# Zip이 처음 보는 값이라 매핑이 안되면(NaN), 평균 사기율로 채움
df['zip_risk'] = df['zip_risk'].fillna(df['is_fraud'].mean())

print("✅ Feature Engineering Complete")

⏳ Engineering Sniper Features...
✅ Feature Engineering Complete


In [ ]:
# 모델에 들어갈 최종 컬럼 선정

"""
tech_mismatch: 작성자님이 제거하려 했던 use_chip과 has_chip을 살려서 **"IC카드인데 마그네틱으로 긁었다"**는 강력한 사기 정황을 잡아냈습니다.

mcc_risk, zip_risk: 수백 개의 카테고리를 One-Hot Encoding 하면 차원이 폭발하지만, "사기 확률(Risk Score)" 하나로 압축하면 차원은 1개로 줄면서 정보량은 극대화됩니다. (트리 모델이 아주 좋아하는 방식입니다.)

utilization_ratio: 단순히 "500불 썼다"보다 "한도 500불인데 500불 썼다(한도 초과 임박)"가 사기꾼의 행동 패턴(Bust-out Fraud)에 더 가깝습니다.

zip의 부활: 위도/경도가 없어서 거리를 못 구한다면, 최소한 **"이 우편번호(동네)에서 사고가 많이 났나?"**라도 파악해야 합니다. zip_risk가 그 역할을 합니다.
"""


features = [
    # 1. 핵심 수치
    'amount', 
    'utilization_ratio',      # 한도 대비 금액
    'amount_income_ratio',    # 소득 대비 금액
    
    # 2. 기술적/패턴 맥락
    'tech_mismatch',          # 칩 카드인데 긁었는지
    'pin_years_gap',          # 비밀번호 변경 오래됐는지
    'num_credit_cards',       # 카드 개수 (다중 발급 사기 연관)
    
    # 3. 시간 맥락
    'hour',
    'is_night',
    
    # 4. 위치/업종 맥락 (Risk Score로 변환됨)
    'mcc_risk',               # 업종 위험도
    'state_risk',             # 주(State) 위험도
    'zip_risk',               # 지역(Zip) 위험도
    
    # 5. 사용자 정보 (보조)
    'current_age',
    'credit_score',           # 신용점수 낮으면 위험할 수 있음

    # 6. 타겟 (정답)
    'is_fraud'
]

df_final = df[features].copy()

# 결측치 최종 확인 (Target Encoding 과정에서 발생할 수 있음)
df_final = df_final.fillna(0)

print(f"🚀 Final Dataset Shape: {df_final.shape}")
print(df_final.head())

# 저장 (선택)
# df_final.to_csv('processed_fraud_data.csv', index=False)

🚀 Final Dataset Shape: (9151537, 14)
   amount  utilization_ratio  amount_income_ratio  tech_mismatch  \
0   33.96           0.001930             0.000570              1   
1    7.78           0.000442             0.000131              1   
2   65.86           0.003742             0.001105              1   
3    1.37           0.000078             0.000023              1   
4  167.62           0.009524             0.002812              0   

   pin_years_gap  num_credit_cards  hour  is_night  mcc_risk  state_risk  \
0              5                 4    13         0  0.000270    0.000035   
1              5                 4    19         0  0.000181    0.000035   
2              5                 4    22         0  0.000817    0.000035   
3              5                 4    15         0  0.000134    0.000035   
4              5                 4     8         0  0.001108    0.008378   

   zip_risk  current_age  credit_score  is_fraud  
0  0.000000           33           763        

# 학습 5

학습 4의 조건을 그대로 활용하되, 2차 정제 데이터를 사용합니다.

결과 : 

threshold = 0.900, precision = 0.7177, recall = 0.5277, TP = 1457, FP = 573

threshold = 0.950, precision = 0.8810, recall = 0.3325, TP = 918, FP = 124

threshold = 0.995, precision = 1.0000, recall = 0.0101, TP = 28, FP = 0

전체적인 탐지성능이 비약적으로 향상되었습니다.

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler

# [Bug Fix] 무한대(inf) 값 처리
# credit_limit가 0인 경우 발생한 inf를 NaN으로 변환하여 XGBoost가 처리할 수 있게 함
df_final = df_final.replace([np.inf, -np.inf], np.nan)

# 혹시 NaN이 생겼다면 0으로 채우기 (선택사항, XGBoost는 NaN도 처리 가능하나 0이 안전함)
df_final = df_final.fillna(0)

print("✅ inf/nan values fixed.")

X = df_final.drop('is_fraud', axis=1)
y = df_final['is_fraud']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. 보수적인 샘플링 (Undersampling Only)
# -----------------------------------------------------------
# 정상:사기 비율을 10:1 (0.1) 정도로 맞춥니다.
# 정상을 너무 많이 버리면 "정상 패턴"을 학습 못해서 오탐(FP)이 늘어납니다.
# 10:1 정도면 모델이 "아, 정상이 훨씬 많구나"라고 인지하여 보수적으로 변합니다.
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(f"Resampled Train shape: {X_resampled.shape}")
print(f"Resampled Fraud ratio: {y_resampled.mean():.4f}") # 약 0.09 (9%)

# 3. 모델 정의 (가중치 제거 & 보수적 설정)
# -----------------------------------------------------------
# (1) XGBoost: scale_pos_weight 제거 (기본값 1)
xgb_clf = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,        # 깊이를 줄여서 과적합 방지
    scale_pos_weight=1, # [핵심] 가중치를 주지 않음 -> Precision 중시
    eval_metric='auc',
    n_jobs=-1,
    random_state=42
)

# (2) Random Forest: Precision 확보에 유리
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=12,       # 너무 깊지 않게
    n_jobs=-1,
    random_state=42
)

# 4. 앙상블 학습
# -----------------------------------------------------------
voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('rf', rf_clf)],
    voting='soft'
)

print("🚀 스나이퍼 모델 학습 시작...")
voting_clf.fit(X_resampled, y_resampled)

# 5. 최적의 Threshold 탐색
# -----------------------------------------------------------
# 검증 데이터에 대한 확률 예측
y_prob = voting_clf.predict_proba(X_test)[:, 1]

print("\n=== 🎯 Threshold Tuning for Precision >= 0.99 ===")
print(f"{'Threshold':<10} | {'Precision':<10} | {'Recall':<10} | {'FP Count':<10} | {'TP Count':<10}")
print("-" * 60)

best_th = 0.5
found = False

# 0.5부터 0.999까지 촘촘하게 검사
thresholds = np.arange(0.5, 1.0, 0.05).tolist() + [0.95, 0.96, 0.97, 0.98, 0.99, 0.995, 0.999]
thresholds = sorted(list(set(thresholds)))

for th in thresholds:
    y_pred_th = (y_prob >= th).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_th).ravel()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    print(f"{th:<10.3f} | {precision:<10.4f} | {recall:<10.4f} | {fp:<10} | {tp:<10}")
    
    # 목표 달성 조건 확인 (Precision >= 0.99 이면서 Recall이 0이 아닌 경우)
    if precision >= 0.99 and recall > 0.0:
        if not found: # 조건을 만족하는 첫 번째(가장 낮은) Threshold 저장
            best_th = th
            found = True
        # 더 높은 Recall을 찾기 위해 best_th를 계속 갱신할 수도 있음 (여기서는 첫 달성 지점 기준)

print("-" * 60)

if found:
    print(f"\n✅ 목표 달성! 추천 Threshold: {best_th}")
    
    # 최종 결과 출력
    final_pred = (y_prob >= best_th).astype(int)
    print(f"\n=== Final Report (Threshold: {best_th}) ===")
    print(confusion_matrix(y_test, final_pred))
    print(classification_report(y_test, final_pred))
else:
    print("\n⚠️ Precision 0.99를 달성하지 못했습니다. 더 강력한 Feature가 필요합니다.")

✅ inf/nan values fixed.
Resampled Train shape: (121462, 13)
Resampled Fraud ratio: 0.0909
🚀 스나이퍼 모델 학습 시작...

=== 🎯 Threshold Tuning for Precision >= 0.99 ===
Threshold  | Precision  | Recall     | FP Count   | TP Count  
------------------------------------------------------------
0.500      | 0.1193     | 0.9265     | 18886      | 2558      
0.550      | 0.1415     | 0.9022     | 15117      | 2491      
0.600      | 0.1754     | 0.8703     | 11294      | 2403      
0.650      | 0.2285     | 0.8385     | 7816       | 2315      
0.700      | 0.3096     | 0.8066     | 4967       | 2227      
0.750      | 0.4006     | 0.7733     | 3195       | 2135      
0.800      | 0.5012     | 0.7280     | 2000       | 2010      
0.850      | 0.6139     | 0.6537     | 1135       | 1805      
0.900      | 0.7177     | 0.5277     | 573        | 1457      
0.950      | 0.8810     | 0.3325     | 124        | 918       
0.950      | 0.8810     | 0.3325     | 124        | 918       
0.960      | 0.9140     

# 3차 데이터 정제

2차 정제 데이터의 효과가 좋았으므로 "맥락"에 관련된 정보를 더 추가합니다.

특히 기존엔 넣지 않은 "시간"에 대한 정보를 추가합니다.

'time_diff_seconds', # 직전 거래와의 시간(초) 차이

'count_24h', # 지난 24시간 동안 거래 횟수

'sum_amt_1h', # 지난 1시간 동안 결제 금액 합계

In [4]:
# 시간벡터 생성
# df_final을 만들기 전, 기초 정제가 끝난 df 상태에서 수행해야 합니다.
# (Users, Cards가 병합되고, date가 datetime으로 변환된 상태)

print("⏳ Generating Velocity Features...")

# 1. 정렬 (User별, 시간순)
# 시간 차이를 계산하려면 반드시 정렬되어 있어야 합니다.
df['date_obj'] = pd.to_datetime(df['date']) # 만약 안 되어 있다면
df = df.sort_values(['client_id', 'date_obj'])

# 2. [핵심 1] 직전 거래와의 시간 차이 (Time Since Last Transaction)
# 사기꾼은 짧은 시간에 여러 번 긁습니다.
# client_id별로 그룹화하여 diff 계산 (초 단위)
df['time_diff_seconds'] = df.groupby('client_id')['date_obj'].diff().dt.total_seconds()
df['time_diff_seconds'] = df['time_diff_seconds'].fillna(999999) # 첫 거래는 큰 값으로

# 3. [핵심 2] 지난 24시간 동안 거래 횟수 (Velocity Count)
# "지난 24시간 동안 100번 긁었다" -> 사기일 확률 매우 높음
# set_index 후 rolling을 쓰면 빠릅니다.
df_sorted = df.set_index('date_obj').sort_index()
# client_id별로 24시간 window count
# (데이터가 커서 오래 걸리면 1시간(1h)으로 줄여도 좋습니다)
count_24h = df_sorted.groupby('client_id')['amount'].rolling('24h').count()

# 인덱스 리셋 후 원본에 병합하기 위해 정리
count_24h = count_24h.reset_index()
# 컬럼명 정리 (client_id, date_obj, amount -> count_24h)
count_24h.columns = ['client_id', 'date_obj', 'count_24h']

# 원본 df에 병합 (Key: client_id, date_obj)
# 주의: 중복 timestamp가 있을 수 있으니 drop_duplicates 고려
df = pd.merge(df, count_24h, on=['client_id', 'date_obj'], how='left')

# 4. [핵심 3] 지난 1시간 동안 결제 금액 합계 (Velocity Amount)
# "평소 100불 쓰는데 지난 1시간 동안 5000불 썼다"
amt_1h = df_sorted.groupby('client_id')['amount'].rolling('1h').sum()
amt_1h = amt_1h.reset_index()
amt_1h.columns = ['client_id', 'date_obj', 'sum_amt_1h']

df = pd.merge(df, amt_1h, on=['client_id', 'date_obj'], how='left')

# 5. 기존 파생변수와 결합
# 이전에 만든 tech_mismatch, utilization_ratio 등도 당연히 포함해야 합니다.

print("✅ Velocity Features Generated")

⏳ Generating Velocity Features...
✅ Velocity Features Generated


In [5]:
# 모델에 들어갈 최종 컬럼 선정

"""
tech_mismatch: 작성자님이 제거하려 했던 use_chip과 has_chip을 살려서 **"IC카드인데 마그네틱으로 긁었다"**는 강력한 사기 정황을 잡아냈습니다.

mcc_risk, zip_risk: 수백 개의 카테고리를 One-Hot Encoding 하면 차원이 폭발하지만, "사기 확률(Risk Score)" 하나로 압축하면 차원은 1개로 줄면서 정보량은 극대화됩니다. (트리 모델이 아주 좋아하는 방식입니다.)

utilization_ratio: 단순히 "500불 썼다"보다 "한도 500불인데 500불 썼다(한도 초과 임박)"가 사기꾼의 행동 패턴(Bust-out Fraud)에 더 가깝습니다.

zip의 부활: 위도/경도가 없어서 거리를 못 구한다면, 최소한 **"이 우편번호(동네)에서 사고가 많이 났나?"**라도 파악해야 합니다. zip_risk가 그 역할을 합니다.
"""


features = [
    # 1. 핵심 수치
    'amount', 
    'utilization_ratio',      # 한도 대비 금액
    'amount_income_ratio',    # 소득 대비 금액
    
    # 2. 기술적/패턴 맥락
    'tech_mismatch',          # 칩 카드인데 긁었는지
    'pin_years_gap',          # 비밀번호 변경 오래됐는지
    'num_credit_cards',       # 카드 개수 (다중 발급 사기 연관)
    
    # 3. 시간 맥락
    'hour',
    'is_night',
    
    # 4. 위치/업종 맥락 (Risk Score로 변환됨)
    'mcc_risk',               # 업종 위험도
    'state_risk',             # 주(State) 위험도
    'zip_risk',               # 지역(Zip) 위험도
    
    # 5. 사용자 정보 (보조)
    'current_age',
    'credit_score',           # 신용점수 낮으면 위험할 수 있음

    # 6. 시간벡터
    'time_diff_seconds',
    'count_24h',
    'sum_amt_1h',

    # 7. 타겟 (정답)
    'is_fraud'
]

df_final = df[features].copy()

# 결측치 최종 확인 (Target Encoding 과정에서 발생할 수 있음)
df_final = df_final.fillna(0)

print(f"🚀 Final Dataset Shape: {df_final.shape}")
print(df_final.head())

# 저장 (선택)
# df_final.to_csv('processed_fraud_data.csv', index=False)


🚀 Final Dataset Shape: (9151537, 17)
   amount  utilization_ratio  amount_income_ratio  tech_mismatch  \
0   33.96           0.001930             0.000570              1   
1    7.78           0.000442             0.000131              1   
2   65.86           0.003742             0.001105              1   
3    1.37           0.000078             0.000023              1   
4  167.62           0.009524             0.002812              0   

   pin_years_gap  num_credit_cards  hour  is_night  mcc_risk  state_risk  \
0              5                 4    13         0  0.000270    0.000035   
1              5                 4    19         0  0.000181    0.000035   
2              5                 4    22         0  0.000817    0.000035   
3              5                 4    15         0  0.000134    0.000035   
4              5                 4     8         0  0.001108    0.008378   

   zip_risk  current_age  credit_score  time_diff_seconds  count_24h  \
0  0.000000           33 

# 학습 6

학습 4, 학습 5와 동일하되, 3차 정제 데이터를 사용합니다.

결과 : 

threshold = 0.900, precision = 0.7432, recall = 0.5683, FP = 542, TP = 1569

threshold = 0.950, precision = 0.8619, recall = 0.3865, FP = 171, TP = 1067

threshold = 0.995, precision = 1.0000, recall = 0.0076, FP = 0, TP = 21

시간벡터 적용 후 0.900, 0.950 임계값에서 판별력이 더욱 향상 되었습니다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler

# [Bug Fix] 무한대(inf) 값 처리
# credit_limit가 0인 경우 발생한 inf를 NaN으로 변환하여 XGBoost가 처리할 수 있게 함
df_final = df_final.replace([np.inf, -np.inf], np.nan)

# 혹시 NaN이 생겼다면 0으로 채우기 (선택사항, XGBoost는 NaN도 처리 가능하나 0이 안전함)
df_final = df_final.fillna(0)

print("✅ inf/nan values fixed.")

X = df_final.drop('is_fraud', axis=1)
y = df_final['is_fraud']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. 보수적인 샘플링 (Undersampling Only)
# -----------------------------------------------------------
# 정상:사기 비율을 10:1 (0.1) 정도로 맞춥니다.
# 정상을 너무 많이 버리면 "정상 패턴"을 학습 못해서 오탐(FP)이 늘어납니다.
# 10:1 정도면 모델이 "아, 정상이 훨씬 많구나"라고 인지하여 보수적으로 변합니다.
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(f"Resampled Train shape: {X_resampled.shape}")
print(f"Resampled Fraud ratio: {y_resampled.mean():.4f}") # 약 0.09 (9%)

# 3. 모델 정의 (가중치 제거 & 보수적 설정)
# -----------------------------------------------------------
# (1) XGBoost: scale_pos_weight 제거 (기본값 1)
xgb_clf = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,        # 깊이를 줄여서 과적합 방지
    scale_pos_weight=1, # [핵심] 가중치를 주지 않음 -> Precision 중시
    eval_metric='auc',
    n_jobs=-1,
    random_state=42
)

# (2) Random Forest: Precision 확보에 유리
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=12,       # 너무 깊지 않게
    n_jobs=-1,
    random_state=42
)

# 4. 앙상블 학습
# -----------------------------------------------------------
voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('rf', rf_clf)],
    voting='soft'
)

print("🚀 스나이퍼 모델 학습 시작...")
voting_clf.fit(X_resampled, y_resampled)

# 5. 최적의 Threshold 탐색
# -----------------------------------------------------------
# 검증 데이터에 대한 확률 예측
y_prob = voting_clf.predict_proba(X_test)[:, 1]

print("\n=== 🎯 Threshold Tuning for Precision >= 0.99 ===")
print(f"{'Threshold':<10} | {'Precision':<10} | {'Recall':<10} | {'FP Count':<10} | {'TP Count':<10}")
print("-" * 60)

best_th = 0.5
found = False

# 0.5부터 0.999까지 촘촘하게 검사
thresholds = np.arange(0.5, 1.0, 0.05).tolist() + [0.95, 0.96, 0.97, 0.98, 0.99, 0.995, 0.999]
thresholds = sorted(list(set(thresholds)))

for th in thresholds:
    y_pred_th = (y_prob >= th).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_th).ravel()
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    print(f"{th:<10.3f} | {precision:<10.4f} | {recall:<10.4f} | {fp:<10} | {tp:<10}")
    
    # 목표 달성 조건 확인 (Precision >= 0.99 이면서 Recall이 0이 아닌 경우)
    if precision >= 0.99 and recall > 0.0:
        if not found: # 조건을 만족하는 첫 번째(가장 낮은) Threshold 저장
            best_th = th
            found = True
        # 더 높은 Recall을 찾기 위해 best_th를 계속 갱신할 수도 있음 (여기서는 첫 달성 지점 기준)

print("-" * 60)

if found:
    print(f"\n✅ 목표 달성! 추천 Threshold: {best_th}")
    
    # 최종 결과 출력
    final_pred = (y_prob >= best_th).astype(int)
    print(f"\n=== Final Report (Threshold: {best_th}) ===")
    print(confusion_matrix(y_test, final_pred))
    print(classification_report(y_test, final_pred))
else:
    print("\n⚠️ Precision 0.99를 달성하지 못했습니다. 더 강력한 Feature가 필요합니다.")

✅ inf/nan values fixed.
Resampled Train shape: (121462, 16)
Resampled Fraud ratio: 0.0909
🚀 스나이퍼 모델 학습 시작...

=== 🎯 Threshold Tuning for Precision >= 0.99 ===
Threshold  | Precision  | Recall     | FP Count   | TP Count  
------------------------------------------------------------
0.500      | 0.1377     | 0.9236     | 15969      | 2550      
0.550      | 0.1611     | 0.9091     | 13071      | 2510      
0.600      | 0.1896     | 0.8924     | 10535      | 2464      
0.650      | 0.2268     | 0.8674     | 8163       | 2395      
0.700      | 0.2786     | 0.8359     | 5975       | 2308      
0.750      | 0.3545     | 0.8001     | 4022       | 2209      
0.800      | 0.4622     | 0.7523     | 2417       | 2077      
0.850      | 0.6032     | 0.6827     | 1240       | 1885      
0.900      | 0.7432     | 0.5683     | 542        | 1569      
0.950      | 0.8619     | 0.3865     | 171        | 1067      
0.950      | 0.8619     | 0.3865     | 171        | 1067      
0.960      | 0.8834     

# 학습 7

3차 정제 데이터를 사용하여 다양한 파라미터에 대한 XGBoost 반복학습을 진행합니다.

threshold = 0.995, 0.990에 대하여 한정하고 Precision과 Recall이 최고로 잘 나오는 모델을 찾습니다.

결과 : 

threshold = 0.995, max_depth = 5, learning_rate = 0.05, n_estimators = 200

-> Precision = 0.9207, Recall = 0.0967, TP = 267, FP = 23

threshold = 0.990, max_depth = 5, learning_rate = 0.05, n_estimators = 200

-> Precision = 0.9030, Recall = 0.2763, TP = 763, FP = 82

Precision이 1에 근접하지 않아 XGBoost는 고임계점 활용엔 어렵다고 판단됩니다.

In [39]:
import pandas as pd
import numpy as np
import itertools
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# 1. 데이터 준비 (time_diff_seconds가 포함된 df_final 사용 가정)
# ----------------------------------------------------------------
# X, y 정의
X = df_final.drop('is_fraud', axis=1)
y = df_final['is_fraud']

# Train/Test 분리 (Stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Under Sampling (10:1 비율 유지)
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(f"Dataset Ready. Train: {X_resampled.shape}, Test: {X_test.shape}")

# 2. 튜닝할 파라미터 그리드 설정
# ----------------------------------------------------------------
# 깊이(max_depth)를 늘리면 더 복잡한 사기 패턴(괴랄한 조건)을 외웁니다.
# 학습률(learning_rate)을 줄이고 트리를 늘리면(n_estimators) 더 정교해집니다.
param_grid = {
    'max_depth': [5, 6, 7, 8, 9, 10, 15],          # 깊이 변화 (핵심)
    'learning_rate': [0.05, 0.01],      # 정밀도 변화
    'n_estimators': [200, 300]         # 트리 개수
}

# 모든 조합 생성
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

print(f"🚀 총 {len(combinations)}개의 모델 조합을 테스트합니다...")

# 3. 실험 및 결과 기록
# ----------------------------------------------------------------
results = []

for i, params in enumerate(combinations):
    print(f"[{i+1}/{len(combinations)}] Testing: {params} ...", end=" ")
    
    # 모델 정의 및 학습
    model = XGBClassifier(
        **params,
        scale_pos_weight=1,   # Sniper 모드 (가중치 없음)
        eval_metric='auc',
        n_jobs=-1,
        random_state=42
    )
    
    model.fit(X_resampled, y_resampled)
    
    # 확률 예측
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # 우리가 관심 있는 Threshold 구간별 성과 측정
    # (1) Auto Block 구간 (Threshold 0.99 이상)
    th_block = 0.995
    y_pred_block = (y_prob >= th_block).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_block).ravel()
    prec_block = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec_block = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # (2) Challenge 구간 (Threshold 0.90 이상)
    th_chal = 0.990
    y_pred_chal = (y_prob >= th_chal).astype(int)
    tn2, fp2, fn2, tp2 = confusion_matrix(y_test, y_pred_chal).ravel()
    prec_chal = tp2 / (tp2 + fp2) if (tp2 + fp2) > 0 else 0
    rec_chal = tp2 / (tp2 + fn2) if (tp2 + fn2) > 0 else 0
    
    # 결과 저장
    res = params.copy()
    
    # Block Tier 성과
    res['Block_Prec'] = round(prec_block, 4)
    res['Block_Recall'] = round(rec_block, 4)
    res['Block_TP'] = tp  # 잡은 사기꾼 수
    res['Block_FP'] = fp  # 억울한 차단 수
    
    # Challenge Tier 성과
    res['Chal_Prec'] = round(prec_chal, 4)
    res['Chal_Recall'] = round(rec_chal, 4)
    res['Chal_TP'] = tp2  # 잡은 사기꾼 수
    res['Chal_FP'] = fp2  # 억울한 차단 수
    
    results.append(res)
    print("Done.")

# 4. 결과 분석 및 출력
# ----------------------------------------------------------------
results_df = pd.DataFrame(results)

# 보기 좋게 정렬: 
# 1순위: Block Precision이 0.99 이상인 것 중에서
# 2순위: Block Recall이 높은 순서대로
results_df = results_df.sort_values(by=['Block_Prec', 'Block_Recall'], ascending=[False, False])

print("\n=== 🎯 Sniper Grid Search Results (Top 5 by Block) ===")
print(results_df.head(10).to_string())


results_df = results_df.sort_values(by=['Chal_Prec', 'Chal_Recall'], ascending=[False, False])

print("\n=== 🎯 Sniper Grid Search Results (Top 5 by Chal) ===")
print(results_df.head(10).to_string())

# CSV 저장 (선택)
# results_df.to_csv('sniper_tuning_results.csv', index=False)

Dataset Ready. Train: (121462, 16), Test: (1830308, 16)
🚀 총 28개의 모델 조합을 테스트합니다...
[1/28] Testing: {'max_depth': 5, 'learning_rate': 0.05, 'n_estimators': 200} ... Done.
[2/28] Testing: {'max_depth': 5, 'learning_rate': 0.05, 'n_estimators': 300} ... Done.
[3/28] Testing: {'max_depth': 5, 'learning_rate': 0.01, 'n_estimators': 200} ... Done.
[4/28] Testing: {'max_depth': 5, 'learning_rate': 0.01, 'n_estimators': 300} ... Done.
[5/28] Testing: {'max_depth': 6, 'learning_rate': 0.05, 'n_estimators': 200} ... Done.
[6/28] Testing: {'max_depth': 6, 'learning_rate': 0.05, 'n_estimators': 300} ... Done.
[7/28] Testing: {'max_depth': 6, 'learning_rate': 0.01, 'n_estimators': 200} ... Done.
[8/28] Testing: {'max_depth': 6, 'learning_rate': 0.01, 'n_estimators': 300} ... Done.
[9/28] Testing: {'max_depth': 7, 'learning_rate': 0.05, 'n_estimators': 200} ... Done.
[10/28] Testing: {'max_depth': 7, 'learning_rate': 0.05, 'n_estimators': 300} ... Done.
[11/28] Testing: {'max_depth': 7, 'learning_rat

# 학습 8

3차 정제 데이터를 사용하여 다양한 파라미터에 대한 RandomForest 반복학습을 진행합니다.

threshold = 0.995, 0.990에 대하여 한정하고 Recall이 최고로 잘 나오는 모델을 찾습니다.

결과 : 

threshold = 0.995, max_depth = 14, n_estimators = 300

-> Precision = 1.0, Recall = 0.00065, TP = 18, FP = 0

threshold = 0.990, max_depth = 13, n_estimators = 300

-> Precision = 1.0, Recall = 0.0116, TP = 32, FP = 0

Recall을 1%까지 끌어올렸습니다.

학습 결과를 보니 고임계점 (고위험군 탐지) 에 대해서는 RandomForest가 XGBoost보다 적합하다고 판단됩니다.

임계점을 좀 더 낮추고 깊이범위를 바꿔서 최적의 모델을 탐색해야 합니다.

In [40]:
import pandas as pd
import numpy as np
import itertools
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# 1. 데이터 준비 (time_diff_seconds가 포함된 df_final 사용 가정)
# ----------------------------------------------------------------
# X, y 정의
X = df_final.drop('is_fraud', axis=1)
y = df_final['is_fraud']

# Train/Test 분리 (Stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Under Sampling (10:1 비율 유지)
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(f"Dataset Ready. Train: {X_resampled.shape}, Test: {X_test.shape}")

# 2. 튜닝할 파라미터 그리드 설정
# ----------------------------------------------------------------
# 깊이(max_depth)를 늘리면 더 복잡한 사기 패턴(괴랄한 조건)을 외웁니다.
# 학습률(learning_rate)을 줄이고 트리를 늘리면(n_estimators) 더 정교해집니다.
param_grid = {
    'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],          # 깊이 변화 (핵심)
    'n_estimators': [100, 200, 300]         # 트리 개수
}

# 모든 조합 생성
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

print(f"🚀 총 {len(combinations)}개의 모델 조합을 테스트합니다...")

# 3. 실험 및 결과 기록
# ----------------------------------------------------------------
results = []

for i, params in enumerate(combinations):
    print(f"[{i+1}/{len(combinations)}] Testing: {params} ...", end=" ")

    model = RandomForestClassifier(
        **params,
        n_jobs=-1,
        random_state=42
    )
    
    model.fit(X_resampled, y_resampled)
    
    # 확률 예측
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # 우리가 관심 있는 Threshold 구간별 성과 측정
    # (1) Auto Block 구간 (Threshold 0.99 이상)
    th_block = 0.995
    y_pred_block = (y_prob >= th_block).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_block).ravel()
    prec_block = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec_block = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # (2) Challenge 구간 (Threshold 0.90 이상)
    th_chal = 0.990
    y_pred_chal = (y_prob >= th_chal).astype(int)
    tn2, fp2, fn2, tp2 = confusion_matrix(y_test, y_pred_chal).ravel()
    prec_chal = tp2 / (tp2 + fp2) if (tp2 + fp2) > 0 else 0
    rec_chal = tp2 / (tp2 + fn2) if (tp2 + fn2) > 0 else 0
    
    # 결과 저장
    res = params.copy()
    
    # Block Tier 성과
    res['Block_Prec'] = round(prec_block, 4)
    res['Block_Recall'] = round(rec_block, 4)
    res['Block_TP'] = tp  # 잡은 사기꾼 수
    res['Block_FP'] = fp  # 억울한 차단 수
    
    # Challenge Tier 성과
    res['Chal_Prec'] = round(prec_chal, 4)
    res['Chal_Recall'] = round(rec_chal, 4)
    res['Chal_TP'] = tp2  # 잡은 사기꾼 수
    res['Chal_FP'] = fp2  # 억울한 차단 수
    
    results.append(res)
    print("Done.")

# 4. 결과 분석 및 출력
# ----------------------------------------------------------------
results_df = pd.DataFrame(results)

# 보기 좋게 정렬: 
# 1순위: Block Precision이 0.99 이상인 것 중에서
# 2순위: Block Recall이 높은 순서대로
results_df = results_df.sort_values(by=['Block_Prec', 'Block_Recall'], ascending=[False, False])

print("\n=== 🎯 Sniper Grid Search Results (Top 5 by Block) ===")
print(results_df.head(10).to_string())


results_df = results_df.sort_values(by=['Chal_Prec', 'Chal_Recall'], ascending=[False, False])

print("\n=== 🎯 Sniper Grid Search Results (Top 5 by Chal) ===")
print(results_df.head(10).to_string())

# CSV 저장 (선택)
# results_df.to_csv('sniper_tuning_results.csv', index=False)

Dataset Ready. Train: (121462, 16), Test: (1830308, 16)
🚀 총 33개의 모델 조합을 테스트합니다...
[1/33] Testing: {'max_depth': 5, 'n_estimators': 100} ... Done.
[2/33] Testing: {'max_depth': 5, 'n_estimators': 200} ... Done.
[3/33] Testing: {'max_depth': 5, 'n_estimators': 300} ... Done.
[4/33] Testing: {'max_depth': 6, 'n_estimators': 100} ... Done.
[5/33] Testing: {'max_depth': 6, 'n_estimators': 200} ... Done.
[6/33] Testing: {'max_depth': 6, 'n_estimators': 300} ... Done.
[7/33] Testing: {'max_depth': 7, 'n_estimators': 100} ... Done.
[8/33] Testing: {'max_depth': 7, 'n_estimators': 200} ... Done.
[9/33] Testing: {'max_depth': 7, 'n_estimators': 300} ... Done.
[10/33] Testing: {'max_depth': 8, 'n_estimators': 100} ... Done.
[11/33] Testing: {'max_depth': 8, 'n_estimators': 200} ... Done.
[12/33] Testing: {'max_depth': 8, 'n_estimators': 300} ... Done.
[13/33] Testing: {'max_depth': 9, 'n_estimators': 100} ... Done.
[14/33] Testing: {'max_depth': 9, 'n_estimators': 200} ... Done.
[15/33] Testing: 

# 학습 9

학습 8과 동일한 조건하에 임계점만 수정합니다.

threshold = 0.985, 0.980에 대하여 한정하고 Recall이 최고로 잘 나오는 모델을 찾습니다.

결과 : 

threshold = 0.985, max_depth = 12, n_estimators = 300

-> Precision = 1.0, Recall = 0.0159, TP = 44, FP = 0

threshold = 0.980, max_depth = 10, n_estimators = 100

-> Precision = 1.0, Recall = 0.0141, TP = 39, FP = 0

현재 방법으로는 Recall의 변화추이를 분석하기 어렵다고 판단됩니다.

threshold 범위를 0.95까지 대폭 늘리고 max_depth 범위도 크게 늘려서 다중 학습이 필요합니다.

In [41]:
import pandas as pd
import numpy as np
import itertools
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# 1. 데이터 준비 (time_diff_seconds가 포함된 df_final 사용 가정)
# ----------------------------------------------------------------
# X, y 정의
X = df_final.drop('is_fraud', axis=1)
y = df_final['is_fraud']

# Train/Test 분리 (Stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Under Sampling (10:1 비율 유지)
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(f"Dataset Ready. Train: {X_resampled.shape}, Test: {X_test.shape}")

# 2. 튜닝할 파라미터 그리드 설정
# ----------------------------------------------------------------
# 깊이(max_depth)를 늘리면 더 복잡한 사기 패턴(괴랄한 조건)을 외웁니다.
# 학습률(learning_rate)을 줄이고 트리를 늘리면(n_estimators) 더 정교해집니다.
param_grid = {
    'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],          # 깊이 변화 (핵심)
    'n_estimators': [100, 200, 300]         # 트리 개수
}

# 모든 조합 생성
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

print(f"🚀 총 {len(combinations)}개의 모델 조합을 테스트합니다...")

# 3. 실험 및 결과 기록
# ----------------------------------------------------------------
results = []

for i, params in enumerate(combinations):
    print(f"[{i+1}/{len(combinations)}] Testing: {params} ...", end=" ")

    model = RandomForestClassifier(
        **params,
        n_jobs=-1,
        random_state=42
    )
    
    model.fit(X_resampled, y_resampled)
    
    # 확률 예측
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # 우리가 관심 있는 Threshold 구간별 성과 측정
    # (1) Auto Block 구간 (Threshold 0.99 이상)
    th_block = 0.985
    y_pred_block = (y_prob >= th_block).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_block).ravel()
    prec_block = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec_block = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    # (2) Challenge 구간 (Threshold 0.90 이상)
    th_chal = 0.980
    y_pred_chal = (y_prob >= th_chal).astype(int)
    tn2, fp2, fn2, tp2 = confusion_matrix(y_test, y_pred_chal).ravel()
    prec_chal = tp2 / (tp2 + fp2) if (tp2 + fp2) > 0 else 0
    rec_chal = tp2 / (tp2 + fn2) if (tp2 + fn2) > 0 else 0
    
    # 결과 저장
    res = params.copy()
    
    # Block Tier 성과
    res['Block_Prec'] = round(prec_block, 4)
    res['Block_Recall'] = round(rec_block, 4)
    res['Block_TP'] = tp  # 잡은 사기꾼 수
    res['Block_FP'] = fp  # 억울한 차단 수
    
    # Challenge Tier 성과
    res['Chal_Prec'] = round(prec_chal, 4)
    res['Chal_Recall'] = round(rec_chal, 4)
    res['Chal_TP'] = tp2  # 잡은 사기꾼 수
    res['Chal_FP'] = fp2  # 억울한 차단 수
    
    results.append(res)
    print("Done.")

# 4. 결과 분석 및 출력
# ----------------------------------------------------------------
results_df = pd.DataFrame(results)

# 보기 좋게 정렬: 
# 1순위: Block Precision이 0.99 이상인 것 중에서
# 2순위: Block Recall이 높은 순서대로
results_df = results_df.sort_values(by=['Block_Prec', 'Block_Recall'], ascending=[False, False])

print("\n=== 🎯 Sniper Grid Search Results (Top 5 by Block) ===")
print(results_df.head(10).to_string())


results_df = results_df.sort_values(by=['Chal_Prec', 'Chal_Recall'], ascending=[False, False])

print("\n=== 🎯 Sniper Grid Search Results (Top 5 by Chal) ===")
print(results_df.head(10).to_string())

# CSV 저장 (선택)
# results_df.to_csv('sniper_tuning_results.csv', index=False)

Dataset Ready. Train: (121462, 16), Test: (1830308, 16)
🚀 총 30개의 모델 조합을 테스트합니다...
[1/30] Testing: {'max_depth': 10, 'n_estimators': 100} ... Done.
[2/30] Testing: {'max_depth': 10, 'n_estimators': 200} ... Done.
[3/30] Testing: {'max_depth': 10, 'n_estimators': 300} ... Done.
[4/30] Testing: {'max_depth': 11, 'n_estimators': 100} ... Done.
[5/30] Testing: {'max_depth': 11, 'n_estimators': 200} ... Done.
[6/30] Testing: {'max_depth': 11, 'n_estimators': 300} ... Done.
[7/30] Testing: {'max_depth': 12, 'n_estimators': 100} ... Done.
[8/30] Testing: {'max_depth': 12, 'n_estimators': 200} ... Done.
[9/30] Testing: {'max_depth': 12, 'n_estimators': 300} ... Done.
[10/30] Testing: {'max_depth': 13, 'n_estimators': 100} ... Done.
[11/30] Testing: {'max_depth': 13, 'n_estimators': 200} ... Done.
[12/30] Testing: {'max_depth': 13, 'n_estimators': 300} ... Done.
[13/30] Testing: {'max_depth': 14, 'n_estimators': 100} ... Done.
[14/30] Testing: {'max_depth': 14, 'n_estimators': 200} ... Done.
[15

# 학습 10

3차 정제 데이터를 활용하고 다양한 파라미터에 대한 RandomForest 반복학습을 진행합니다.

저임계점(2차 인증 대상) 구간의 추이를 분석하는 것이 목표입니다.

threshold = 0.9에 대하여 F1-Score가 최고로 잘 나오는 모델을 찾습니다.

결과 : 

threshold = 0.9, max_depth = 18, n_estimators = 300

-> Precision = 0.7124, Recall = 0.5795, TP = 1600, FP = 646, F1-score = 0.6391

유의미한 모델이 탄생했습니다. Recall과 F1-score을 동시에 더 끌어올릴 방안을 시도해봐야 합니다.

저임계점(2차 인증 대상) 구간에서 XGBoost가 어느정도 위력을 가지는지 재탐색이 필요합니다.

In [37]:
import pandas as pd
import numpy as np
import itertools
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# 1. 데이터 준비 (time_diff_seconds가 포함된 df_final 사용 가정)
# ----------------------------------------------------------------
# X, y 정의
X = df_final.drop('is_fraud', axis=1)
y = df_final['is_fraud']

# Train/Test 분리 (Stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Under Sampling (10:1 비율 유지)
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(f"Dataset Ready. Train: {X_resampled.shape}, Test: {X_test.shape}")

# 2. 튜닝할 파라미터 그리드 설정
# ----------------------------------------------------------------
# 깊이(max_depth)를 늘리면 더 복잡한 사기 패턴(괴랄한 조건)을 외웁니다.
# 학습률(learning_rate)을 줄이고 트리를 늘리면(n_estimators) 더 정교해집니다.
param_grid = {
    'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],          # 깊이 변화 (핵심)
    'n_estimators': [100, 200, 300]         # 트리 개수
}

# 모든 조합 생성
keys, values = zip(*param_grid.items())
combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

print(f"🚀 총 {len(combinations)}개의 모델 조합을 테스트합니다...")

# 3. 실험 및 결과 기록
# ----------------------------------------------------------------
results = []

for i, params in enumerate(combinations):
    print(f"[{i+1}/{len(combinations)}] Testing: {params} ...", end=" ")

    model = RandomForestClassifier(
        **params,
        n_jobs=-1,
        random_state=42
    )
    
    model.fit(X_resampled, y_resampled)
    
    # 확률 예측
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # 우리가 관심 있는 Threshold 구간별 성과 측정
    # (1) Auto Block 구간 (Threshold 0.99 이상)
    th_block = 0.9
    y_pred_block = (y_prob >= th_block).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_block).ravel()
    prec_block = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec_block = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_block = f1_score(y_test, y_pred_block)
    
    # (2) Challenge 구간 (Threshold 0.90 이상)
    th_chal = 0.95
    y_pred_chal = (y_prob >= th_chal).astype(int)
    tn2, fp2, fn2, tp2 = confusion_matrix(y_test, y_pred_chal).ravel()
    prec_chal = tp2 / (tp2 + fp2) if (tp2 + fp2) > 0 else 0
    rec_chal = tp2 / (tp2 + fn2) if (tp2 + fn2) > 0 else 0
    f1_chal = f1_score(y_test, y_pred_chal)
    
    # 결과 저장
    res = params.copy()
    
    # Block Tier 성과
    res['90_Prec'] = round(prec_block, 4)
    res['90_Recall'] = round(rec_block, 4)
    res['90_TP'] = tp  # 잡은 사기꾼 수
    res['90_FP'] = fp  # 억울한 차단 수
    res['90_F1'] = round(f1_block, 4)
    
    # Challenge Tier 성과
    res['95_Prec'] = round(prec_chal, 4)
    res['95_Recall'] = round(rec_chal, 4)
    res['95_TP'] = tp2  # 잡은 사기꾼 수
    res['95_FP'] = fp2  # 억울한 차단 수
    res['95_F1'] = round(f1_chal, 4)
    
    results.append(res)
    print("Done.")

# 4. 결과 분석 및 출력
# ----------------------------------------------------------------
results_df = pd.DataFrame(results)

# 보기 좋게 정렬: 
# 1순위: Block Precision이 0.99 이상인 것 중에서
# 2순위: Block Recall이 높은 순서대로
results_df = results_df.sort_values(by=['90_F1', '95_F1'], ascending=[False, False])

print("\n=== 🎯 Sniper Grid Search Results (Top 5) ===")
print(results_df.head(10).to_string())

# CSV 저장 (선택)
# results_df.to_csv('sniper_tuning_results.csv', index=False)

Dataset Ready. Train: (121462, 16), Test: (1830308, 16)
🚀 총 33개의 모델 조합을 테스트합니다...
[1/33] Testing: {'max_depth': 10, 'n_estimators': 100} ... Done.
[2/33] Testing: {'max_depth': 10, 'n_estimators': 200} ... Done.
[3/33] Testing: {'max_depth': 10, 'n_estimators': 300} ... Done.
[4/33] Testing: {'max_depth': 11, 'n_estimators': 100} ... Done.
[5/33] Testing: {'max_depth': 11, 'n_estimators': 200} ... Done.
[6/33] Testing: {'max_depth': 11, 'n_estimators': 300} ... Done.
[7/33] Testing: {'max_depth': 12, 'n_estimators': 100} ... Done.
[8/33] Testing: {'max_depth': 12, 'n_estimators': 200} ... Done.
[9/33] Testing: {'max_depth': 12, 'n_estimators': 300} ... Done.
[10/33] Testing: {'max_depth': 13, 'n_estimators': 100} ... Done.
[11/33] Testing: {'max_depth': 13, 'n_estimators': 200} ... Done.
[12/33] Testing: {'max_depth': 13, 'n_estimators': 300} ... Done.
[13/33] Testing: {'max_depth': 14, 'n_estimators': 100} ... Done.
[14/33] Testing: {'max_depth': 14, 'n_estimators': 200} ... Done.
[15

# 학습 11

XGBoost, RandomForest의 심층 분석을 하기 전 새로운 알고리즘을 도입하여 테스트합니다.

3차 정제 데이터를 활용하고 CatBoost를 활용한 3단 Grid 학습을 진행합니다.

CatBoost가 Precision >= 0.99의 초고위험군 탐지에 적합한지 테스트합니다.

결과 : 

모델 탐색 실패. Precision >= 0.99 조건을 없애고 CatBoost 자체의 퍼포먼스를 정밀조사할 필요성이 느껴집니다.

In [7]:
import pandas as pd
import numpy as np
import time
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import RandomUnderSampler

# 1. 데이터 준비 (time_diff_seconds가 포함된 df_final 사용 가정)
# ----------------------------------------------------------------
# 범주형 변수 지정 (CatBoost의 강력한 무기)
# 실제 데이터프레임 컬럼명에 맞춰 수정하세요.
# 예: mcc_risk나 state_risk 대신 원본 mcc, merchant_state를 써도 됩니다.
# 여기서는 수치형으로 변환된 df_final을 쓴다고 가정하므로 빈 리스트로 둡니다.
# 만약 원본 문자열 컬럼을 쓴다면 cat_features = ['merchant_state', 'mcc'] 등으로 지정.
cat_features = [] 

X = df_final.drop('is_fraud', axis=1)
y = df_final['is_fraud']

# Train/Test 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Under Sampling (10:1 비율) - 학습 속도 및 불균형 해소
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(f"Data Ready. Train: {X_resampled.shape}, Test: {X_test.shape}")

# 2. 파라미터 그리드 및 변수 설정
# ----------------------------------------------------------------
# CatBoost는 depth가 16을 넘어가면 GPU 메모리가 터지거나 매우 느려질 수 있습니다.
# 보통 6~12 사이에서 성능이 결정됩니다. 16까지 테스트해봅시다.
depth_list = [6, 8, 10, 12, 14] 
estimators_list = [500, 1000, 1500] # CatBoost는 트리를 많이 심어도 과적합이 적음
threshold_list = np.arange(0.80, 1.00, 0.01)

best_record = {
    'recall': 0,
    'precision': 0,
    'threshold': 0,
    'params': {},
    'model': None
}

total_iter = len(depth_list) * len(estimators_list)
current_iter = 0
start_time = time.time()

print(f"🚀 Sniper Grid Search 시작 (총 {total_iter}회 학습)...")

# 3. Grid Search Loop
# ----------------------------------------------------------------
for d in depth_list:
    for n in estimators_list:
        current_iter += 1
        elapsed = time.time() - start_time
        print(f"[{current_iter}/{total_iter}] Depth: {d}, Est: {n} | Time: {elapsed:.0f}s ...", end="")
        
        # 모델 정의 (GPU 사용)
        model = CatBoostClassifier(
            iterations=n,
            depth=d,
            learning_rate=0.05,
            loss_function='Logloss',
            eval_metric='AUC',
            task_type="GPU",       # GPU 사용 설정
            devices='0',           # 첫 번째 GPU 사용
            random_seed=42,
            verbose=0,             # 로그 출력 끄기
            cat_features=cat_features,
            scale_pos_weight=1     # Sniper 모드 (가중치 없음)
        )
        
        try:
            model.fit(X_resampled, y_resampled)
            
            # 확률 예측
            y_prob = model.predict_proba(X_test)[:, 1]
            
            # Threshold 스캔
            local_best_recall = 0
            local_best_prec = 0
            
            for th in threshold_list:
                y_pred = (y_prob >= th).astype(int)
                tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
                
                # 정밀도와 재현율 계산
                prec = tp / (tp + fp) if (tp + fp) > 0 else 0
                rec = tp / (tp + fn) if (tp + fn) > 0 else 0
                
                # 조건: Precision >= 0.99
                if prec >= 0.99:
                    # Recall 최대화 갱신
                    if rec > best_record['recall']:
                        best_record['recall'] = rec
                        best_record['precision'] = prec
                        best_record['threshold'] = th
                        best_record['params'] = {'depth': d, 'n_estimators': n}
                        best_record['model'] = model # 최고 모델 저장
                    
                    # 현재 모델 내에서 로그용 기록
                    if rec > local_best_recall:
                        local_best_recall = rec
                        local_best_prec = prec
            
            print(f" Best Recall (Prec>=0.99): {local_best_recall:.4f}")
            
        except Exception as e:
            print(f" Error: {e}")

# 4. 최종 결과 리포트
# ----------------------------------------------------------------
print("\n" + "="*50)
print("🏆 Final Sniper Model Found!")
print("="*50)
print(f"Best Params    : {best_record['params']}")
print(f"Best Threshold : {best_record['threshold']:.4f}")
print(f"Precision      : {best_record['precision']:.4f}")
print(f"Recall (Catch) : {best_record['recall']:.4f}")
print("="*50)

# 최종 모델로 Confusion Matrix 확인
if best_record['model']:
    final_prob = best_record['model'].predict_proba(X_test)[:, 1]
    final_pred = (final_prob >= best_record['threshold']).astype(int)
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, final_pred))

Data Ready. Train: (121462, 16), Test: (1830308, 16)
🚀 Sniper Grid Search 시작 (총 15회 학습)...
[1/15] Depth: 6, Est: 500 | Time: 0s ...

Default metric period is 5 because AUC is/are not implemented for GPU


 Best Recall (Prec>=0.99): 0.0000
[2/15] Depth: 6, Est: 1000 | Time: 4s ...

Default metric period is 5 because AUC is/are not implemented for GPU


 Best Recall (Prec>=0.99): 0.0000
[3/15] Depth: 6, Est: 1500 | Time: 11s ...

...

 Best Recall (Prec>=0.99): 0.0000
[14/15] Depth: 14, Est: 1000 | Time: 338s ...

Default metric period is 5 because AUC is/are not implemented for GPU


 Best Recall (Prec>=0.99): 0.0000
[15/15] Depth: 14, Est: 1500 | Time: 502s ...

Default metric period is 5 because AUC is/are not implemented for GPU


 Best Recall (Prec>=0.99): 0.0000

🏆 Final Sniper Model Found!
Best Params    : {}
Best Threshold : 0.0000
Precision      : 0.0000
Recall (Catch) : 0.0000


# 학습 12

CatBoost 퍼포먼스 정밀진단으로, 세 가지 관점에서 분석합니다.

- 0.9 이상 Precision 구간에서 최대 Recall을 Threshold 탐색
- 모델 학습에 최대 기여한 데이터 탐색
- 모델이 1로 예측한 항목에 대한 근거가 확실한지 분석

결과 : 

탐지된 사기 모델들을 보면 모델 학습의 방향 자체는 올바르게 되고 있는 것으로 보입니다. (99.999% 이상 확실한 근거로 올바른 사기거래를 탐지하고 있음)

그럼에도 불구하고 Threshold 임계가 너무 과하게 높고 Recall이 나오지 않습니다.

모델 기여도를 보면 직접 정제한 mcc_risk, zip_risk가 압도적으로 높게 기여하고 있는데, 이는 CatBoost의 장점(범주형 데이터의 맥락파악에 강력하다)을 죽이고 있는 것으로 보입니다.

따라서 데이터 재정제를 통해 Catboost를 활용해볼 필요성이 있습니다.

In [8]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 단일 모델로 빠르게 진단 (Depth 10 정도가 적당)
print("🚀 CatBoost 정밀 진단 시작...")
model = CatBoostClassifier(
    iterations=1000,
    depth=10,
    learning_rate=0.05,
    task_type="GPU",      # GPU 사용
    random_seed=42,
    verbose=0,
    scale_pos_weight=1    # Sniper Mode
)

model.fit(X_resampled, y_resampled)
y_prob = model.predict_proba(X_test)[:, 1]

# 2. 정밀도-재현율(PR) 데이터 추출
precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)

# 3. 정밀도 구간별 최대 Recall 확인
print("\n=== 📊 Precision 구간별 성과 분석 ===")
print(f"{'Min Precision':<15} | {'Max Recall':<10} | {'Threshold':<10}")
print("-" * 45)

target_precs = [0.90, 0.95, 0.98, 0.99, 0.995, 0.999]
for target in target_precs:
    # 해당 정밀도 이상을 만족하는 구간 찾기
    valid_indices = [i for i, p in enumerate(precisions) if p >= target]
    
    if valid_indices:
        # 그 중에서 Recall이 가장 높은 지점
        best_idx = valid_indices[0] # 보통 Threshold가 낮을수록 Recall이 높으므로 첫번째가 Max Recall
        # 정밀도 곡선은 들쑥날쑥할 수 있으므로 valid한 것 중 max recall을 찾음
        valid_recalls = [recalls[i] for i in valid_indices]
        max_recall = max(valid_recalls)
        
        # 해당 Recall을 만드는 Threshold 찾기 (approx)
        # precisions/recalls 길이는 thresholds 길이 + 1
        th_idx = [i for i, r in enumerate(recalls) if r == max_recall and precisions[i] >= target][0]
        th_val = thresholds[min(th_idx, len(thresholds)-1)]
        
        print(f"{target:<15} | {max_recall:<10.4f} | {th_val:<10.4f}")
    else:
        print(f"{target:<15} | {'불가능':<10} | {'-'}")

# 4. Feature Importance 확인 (time_diff가 쓰였는지 확인)
print("\n=== 🌟 Feature Importance Top 5 ===")
feat_imp = pd.DataFrame({
    'feature': X.columns,
    'importance': model.get_feature_importance()
}).sort_values('importance', ascending=False)
print(feat_imp.head(5))

# 5. Top 10 예측 확인 (최상위 오탐 확인)
print("\n=== 🔍 Top 10 의심 거래 분석 ===")
top_indices = y_prob.argsort()[::-1][:10]
print(f"{'Rank':<5} | {'Prob':<10} | {'True Label':<10}")
for i, idx in enumerate(top_indices):
    print(f"{i+1:<5} | {y_prob[idx]:<10.6f} | {y_test.iloc[idx]}")

🚀 CatBoost 정밀 진단 시작...

=== 📊 Precision 구간별 성과 분석 ===
Min Precision   | Max Recall | Threshold 
---------------------------------------------
0.9             | 0.2622     | 0.9989    
0.95            | 0.0605     | 0.9999    
0.98            | 0.0344     | 1.0000    
0.99            | 0.0239     | 1.0000    
0.995           | 0.0239     | 1.0000    
0.999           | 0.0239     | 1.0000    

=== 🌟 Feature Importance Top 5 ===
              feature  importance
8            mcc_risk   16.288216
10           zip_risk   14.047003
6                hour   11.003966
9          state_risk   10.231560
13  time_diff_seconds    6.700268

=== 🔍 Top 10 의심 거래 분석 ===
Rank  | Prob       | True Label
1     | 0.999998   | 1
2     | 0.999996   | 1
3     | 0.999996   | 1
4     | 0.999995   | 1
5     | 0.999995   | 1
6     | 0.999994   | 1
7     | 0.999994   | 1
8     | 0.999993   | 1
9     | 0.999992   | 1
10    | 0.999992   | 1


# 4차 데이터 정제

범주형 데이터 분석에 강한 CatBoost를 위해 데이터를 새로이 정제합니다.

임의로 정제했던 데이터 중 범주형 컬럼은 다시 범주형으로 살려냅니다.

컬럼 순서

'amount', 'utilization_ratio', 'amount_income_ratio', 
'tech_mismatch', 'pin_years_gap', 'num_credit_cards', 
'hour', 'is_night', 
'time_diff_seconds', 'count_24h', 'sum_amt_1h', # Velocity
'current_age', 'credit_score',

[핵심 변경] Risk 변수 대신 원본 범주형 변수 사용

'mcc', # mcc_risk 대신 범주형 그대로 사용

'merchant_state', # state_risk 대신 범주형 그대로 사용

'zip',  # zip_risk 대신 범주형 그대로 사용

'use_chip', # 범주형 그대로 사용

'card_brand', # 범주형 브랜드 정보도 추가

'is_fraud' # Target

In [ ]:
# 1. 원본 범주형 데이터 복구
# df_final에는 risk 점수만 있고 원본 코드가 없을 수 있으니, 
# 전처리 단계의 df에서 가져오거나 다시 합쳐야 합니다.
# (df는 velocity feature까지 다 만들어진 상태라고 가정)

# 사용할 피처 재선정 (Risk 변수 제거 -> 원본 변수 투입)
raw_cat_features = [
    'amount', 'utilization_ratio', 'amount_income_ratio', 
    'tech_mismatch', 'pin_years_gap', 'num_credit_cards', 
    'hour', 'is_night', 
    'time_diff_seconds', 'count_24h', 'sum_amt_1h', # Velocity
    'current_age', 'credit_score',
    
    # [핵심 변경] Risk 변수 대신 원본 범주형 변수 사용
    'mcc',            # mcc_risk 대신 (숫자형이라도 범주형으로 취급)
    'merchant_state', # state_risk 대신 (문자열)
    'zip',            # zip_risk 대신 (숫자형이라도 범주형으로 취급)
    'use_chip',       # tech_mismatch가 있지만 원본도 같이 줘봅니다
    'card_brand',     # 브랜드 정보도 추가
    
    'is_fraud' # Target
]

# 해당 컬럼들로 데이터셋 구성
# (주의: Null 값이 있으면 안 됩니다. 문자열은 'Unknown', 숫자는 -1 등으로 채우세요)
df_raw_cat = df[raw_cat_features].copy()
df_raw_cat['merchant_state'] = df_raw_cat['merchant_state'].fillna('Unknown')
df_raw_cat['card_brand'] = df_raw_cat['card_brand'].fillna('Unknown')
df_raw_cat['mcc'] = df_raw_cat['mcc'].astype(str) # 숫자가 아닌 범주로 인식되게 문자열 변환 추천
df_raw_cat['zip'] = df_raw_cat['zip'].astype(str)

In [6]:
df_raw_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9151537 entries, 0 to 9151536
Data columns (total 19 columns):
 #   Column               Dtype  
---  ------               -----  
 0   amount               float64
 1   utilization_ratio    float64
 2   amount_income_ratio  float64
 3   tech_mismatch        int64  
 4   pin_years_gap        int64  
 5   num_credit_cards     int64  
 6   hour                 int32  
 7   is_night             int64  
 8   time_diff_seconds    float64
 9   count_24h            float64
 10  sum_amt_1h           float64
 11  current_age          int64  
 12  credit_score         int64  
 13  mcc                  object 
 14  merchant_state       object 
 15  zip                  object 
 16  use_chip             object 
 17  card_brand           object 
 18  is_fraud             int64  
dtypes: float64(6), int32(1), int64(7), object(5)
memory usage: 1.3+ GB


# 학습 13

4차 정제 데이터에 대하여 CatBoost 학습을 진행합니다.

Precision = 이 0.99 이상인 모델 중 Recall이 높은 임계점을 실수범위에서 탐색합니다.

결과 :

Threshold : 0.99899755

Precision : 0.9901

Recall    : 0.4335

매우 좋은 모델이 학습되었습니다. 이제 depth를 좀 바꿔가며 최고의 Tier 1 (초고위험군) 모델을 선정하면 최고의 모델 선정이 가능해 보입니다.

In [ ]:
# CatBoost 학습 (Raw Features)
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

X = df_raw_cat.drop('is_fraud', axis=1)
y = df_raw_cat['is_fraud']

# 범주형 변수 리스트 지정 (이게 핵심!)
cat_features_indices = ['mcc', 'merchant_state', 'zip', 'use_chip', 'card_brand']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Undersampling (10:1)
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print("🚀 CatBoost Raw Category 학습 시작...")

model = CatBoostClassifier(
    iterations=1500,        # 넉넉하게
    depth=10,
    learning_rate=0.05,
    task_type="GPU",        # GPU 가속
    eval_metric='AUC',
    cat_features=cat_features_indices, # [핵심] 여기에 원본 컬럼 지정
    random_seed=42,
    verbose=100,
    scale_pos_weight=1
)

model.fit(X_resampled, y_resampled, eval_set=(X_test, y_test))

# 3. 성능 확인 (Recall 체크)
y_prob = model.predict_proba(X_test)[:, 1]

# Precision 0.99 이상인 구간의 Recall 확인
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)

print("\n=== 📊 New CatBoost (Raw Features) 성과 ===")
target_precs = [0.95, 0.98, 0.99]
for target in target_precs:
    valid_indices = [i for i, p in enumerate(precisions) if p >= target]
    if valid_indices:
        valid_recalls = [recalls[i] for i in valid_indices]
        print(f"Precision >= {target}: Max Recall = {max(valid_recalls):.4f}")
    else:
        print(f"Precision >= {target}: 불가능")

🚀 CatBoost Raw Category 학습 시작...


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9767171	best: 0.9767171 (0)	total: 76.7ms	remaining: 1m 55s
100:	test: 0.9952612	best: 0.9952612 (100)	total: 7.5s	remaining: 1m 43s
200:	test: 0.9959113	best: 0.9959113 (200)	total: 15.2s	remaining: 1m 38s
300:	test: 0.9961649	best: 0.9961649 (300)	total: 22.9s	remaining: 1m 31s
400:	test: 0.9962667	best: 0.9962749 (390)	total: 30.7s	remaining: 1m 24s
500:	test: 0.9963647	best: 0.9963713 (490)	total: 38.5s	remaining: 1m 16s
600:	test: 0.9964501	best: 0.9964501 (600)	total: 46.4s	remaining: 1m 9s
700:	test: 0.9964825	best: 0.9964871 (680)	total: 54.1s	remaining: 1m 1s
800:	test: 0.9964259	best: 0.9964871 (680)	total: 1m 1s	remaining: 54.1s
900:	test: 0.9964379	best: 0.9964871 (680)	total: 1m 9s	remaining: 46.3s
1000:	test: 0.9964314	best: 0.9964871 (680)	total: 1m 17s	remaining: 38.6s
1100:	test: 0.9964395	best: 0.9964871 (680)	total: 1m 25s	remaining: 30.9s
1200:	test: 0.9964095	best: 0.9964871 (680)	total: 1m 33s	remaining: 23.2s
1300:	test: 0.9964210	best: 0.9964871 (680)

In [16]:
import numpy as np

# 1. 목표 Precision 설정
target_prec = 0.99

# 2. 조건을 만족하는 인덱스 찾기
# precisions 배열은 thresholds보다 길이가 1 더 깁니다 (마지막 1.0 포함).
# 따라서 길이를 맞추기 위해 마지막 요소 제외하고 비교하거나 인덱싱에 주의해야 합니다.
valid_indices = np.where(precisions[:-1] >= target_prec)[0]

if len(valid_indices) > 0:
    # 3. 그 중에서 Recall이 가장 높은 인덱스 찾기
    # (보통 Threshold가 낮을수록 Recall이 높으므로, valid_indices의 첫 번째가 Recall이 가장 높을 확률이 큼)
    # 하지만 확실하게 하기 위해 argmax를 씁니다.
    best_idx = valid_indices[np.argmax(recalls[valid_indices])]
    
    best_threshold = thresholds[best_idx]
    best_recall = recalls[best_idx]
    best_precision = precisions[best_idx]
    
    print(f"=== 🏆 Best Sniper Option Found ===")
    print(f"Threshold : {best_threshold:.8f}")
    print(f"Precision : {best_precision:.4f}")
    print(f"Recall    : {best_recall:.4f}")
    
    # [Tip] 바로 검증해보기
    print("\n[검증 결과]")
    final_pred = (y_prob >= best_threshold).astype(int)
    from sklearn.metrics import confusion_matrix
    print(confusion_matrix(y_test, final_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, final_pred).ravel()
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0
    rec = tp / (tp + fn) if (tp + fn) > 0 else 0
    print(tp, fp, prec, rec)
    
else:
    print(f"Precision {target_prec} 이상을 만족하는 구간이 없습니다.")

=== 🏆 Best Sniper Option Found ===
Threshold : 0.99899755
Precision : 0.9901
Recall    : 0.4335

[검증 결과]
[[1827535      12]
 [   1564    1197]]
1197 12 0.9900744416873449 0.43353857298080406


In [13]:
# 모델 저장
model.save_model("catboost_sniper_model_recall43.cbm")
print("✅ 모델이 저장되었습니다.")

# 나중에 불러오기
# from catboost import CatBoostClassifier
# loaded_model = CatBoostClassifier()
# loaded_model.load_model("catboost_sniper_model_recall43.cbm")

✅ 모델이 저장되었습니다.


# 학습 14

우선 학습 13에서 찾은 모델을 Tier 1에 배치하고 Tier 2 모델을 찾는 시뮬레이션을 수행합니다.

Tier 2 모델은 F1-score가 높은 모델의 임계점을 실수범위에서 탐색합니다. 

결과 :

Tier 1 (즉시 차단) : Threshold 0.998998
   [성과] 사기꾼 1,197명 검거 (검거율 43.4%)
   [비용] 오탐 12건 (정밀도 0.9901)
          ㄴ 극소수의 VIP 민원 발생 가능 (SLA 허용 범위 내)
          
Tier 2 (추가 인증) : Threshold 0.987650
   [성과] 사기꾼 732명 추가 검거 (추가 검거율 26.5%)
   [비용] 일반인 280명이 인증 수행 (정밀도 0.7233)
          ㄴ 전체 거래의 0.06%만 인증 요청 (사용자 경험 영향 미미)
          
Tier 3 (자동 통과)
   [위험] 놓친 사기꾼 832명 (30.1%)

Tier 1 - Tier 2 로 연계되는 2중 검증 시스템은 유효하다는 판단입니다.

특히 Tier 2의 목적을 Recall에 두느냐, F1-score에 두느냐에 따라 서로 다른 비즈니스적 요구를 만족시킬 수 있습니다.

이제 파라미터값을 변화시키면서 최적의 Tier 1 모델과 Tier 2 모델을 찾아내야 합니다.

In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, f1_score

# [필수] y_test와 y_prob는 메모리에 있어야 합니다.
# (CatBoost 모델 예측 결과)

print("🚀 2-Tier 전략 최종 조율 시작...")

# ---------------------------------------------------------
# 1. Tier 1 (Sniper) 설정 : 찾으신 값으로 고정
# ---------------------------------------------------------
th_tier1 = 0.99899755  # [사용자가 찾은 황금 임계값]

print(f"\n🔫 [Tier 1: Block] 확정 Threshold: {th_tier1}")

# ---------------------------------------------------------
# 2. Tier 2 (Patrol) 탐색 : Tier 1 미만 구간 정밀 스캔
# ---------------------------------------------------------
# 모든 가능한 임계값 후보를 가져옵니다.
precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)

# DataFrame으로 변환
df_pr = pd.DataFrame({
    'threshold': thresholds,
    'precision': precisions[:-1],
    'recall': recalls[:-1]
})

# F1-Score 계산
# (분모가 0일 경우 0으로 처리하여 에러 방지)
df_pr['f1'] = 2 * (df_pr['precision'] * df_pr['recall']) / (df_pr['precision'] + df_pr['recall'] + 1e-10)

# Tier 1보다 낮은 구간만 필터링
candidates = df_pr[df_pr['threshold'] < th_tier1]

# F1-Score가 가장 높은 지점 찾기
best_patrol = candidates.sort_values('f1', ascending=False).iloc[0]
th_tier2 = best_patrol['threshold']

print(f"\n👮 [Tier 2: Challenge] 최적 Threshold Found!")
print(f"   - Threshold: {th_tier2:.8f}")
print(f"   - F1-Score : {best_patrol['f1']:.4f}")
print(f"   - Precision: {best_patrol['precision']:.4f}")
print(f"   - Recall   : {best_patrol['recall']:.4f}")

# ---------------------------------------------------------
# 3. 최종 방어 전략 시뮬레이션
# ---------------------------------------------------------
print("\n" + "="*60)
print("📊 [최종 보고서] FraudGuard 2-Tier Defense Strategy")
print("="*60)

n_total = len(y_test)
n_fraud = sum(y_test)

# 그룹 분류
# 1) Blocked (Tier 1 이상)
mask_block = (y_prob >= th_tier1)
n_blocked = sum(mask_block)
n_blocked_fraud = sum(mask_block & (y_test == 1))
n_blocked_innocent = n_blocked - n_blocked_fraud

# 2) Challenged (Tier 2 이상 ~ Tier 1 미만)
mask_chal = (y_prob >= th_tier2) & (y_prob < th_tier1)
n_chal = sum(mask_chal)
n_chal_fraud = sum(mask_chal & (y_test == 1))
n_chal_innocent = n_chal - n_chal_fraud

# 3) Passed (Tier 2 미만)
mask_pass = (y_prob < th_tier2)
n_passed = sum(mask_pass)
n_passed_fraud = sum(mask_pass & (y_test == 1))

# 주요 지표 계산 (0 나누기 방지)
recall_tier1 = (n_blocked_fraud / n_fraud * 100) if n_fraud > 0 else 0
recall_tier2 = (n_chal_fraud / n_fraud * 100) if n_fraud > 0 else 0
total_recall = ((n_blocked_fraud + n_chal_fraud) / n_fraud * 100) if n_fraud > 0 else 0

prec_tier1 = (n_blocked_fraud / n_blocked) if n_blocked > 0 else 0
prec_tier2 = (n_chal_fraud / n_chal) if n_chal > 0 else 0

# 출력
print(f"전체 거래: {n_total:,}건 (실제 사기: {n_fraud:,}건)")
print("-" * 60)

print(f"🔴 Tier 1 (즉시 차단) : Threshold {th_tier1:.6f}")
print(f"   [성과] 사기꾼 {n_blocked_fraud:,}명 검거 (검거율 {recall_tier1:.1f}%)")
print(f"   [비용] 오탐 {n_blocked_innocent:,}건 (정밀도 {prec_tier1:.4f})")
if n_blocked_innocent > 0:
    print(f"          ㄴ ⚠️ 극소수의 VIP 민원 발생 가능 (SLA 허용 범위 내)")
else:
    print(f"          ㄴ ✅ 오탐 0건! 완벽한 방어")

print("-" * 60)

print(f"🟡 Tier 2 (추가 인증) : Threshold {th_tier2:.6f}")
print(f"   [성과] 사기꾼 {n_chal_fraud:,}명 추가 검거 (추가 검거율 {recall_tier2:.1f}%)")
print(f"   [비용] 일반인 {n_chal_innocent:,}명이 인증 수행 (정밀도 {prec_tier2:.4f})")
print(f"          ㄴ 전체 거래의 {(n_chal/n_total*100):.2f}%만 인증 요청 (사용자 경험 영향 미미)")

print("-" * 60)

print(f"🟢 Tier 3 (자동 통과)")
print(f"   [위험] 놓친 사기꾼 {n_passed_fraud:,}명 ({n_passed_fraud/n_fraud*100:.1f}%)")

print("="*60)
print(f"🏆 프로젝트 최종 성적표")
print(f"   1. 총 사기 방어율 (Recall) : {total_recall:.1f}%")
print(f"   2. 자동 차단 정밀도        : {prec_tier1:.4f} (목표: 0.99)")
print(f"   3. 2차 방어선 효율         : 일반인 1명 귀찮게 할 때마다 사기꾼 {prec_tier2/(1-prec_tier2):.1f}명 잡음")
print("="*60)

🚀 2-Tier 전략 최종 조율 시작...

🔫 [Tier 1: Block] 확정 Threshold: 0.99899755

👮 [Tier 2: Challenge] 최적 Threshold Found!
   - Threshold: 0.98764950
   - F1-Score : 0.7744
   - Precision: 0.8685
   - Recall   : 0.6987

📊 [최종 보고서] FraudGuard 2-Tier Defense Strategy
전체 거래: 1,830,308건 (실제 사기: 2,761건)
------------------------------------------------------------
🔴 Tier 1 (즉시 차단) : Threshold 0.998998
   [성과] 사기꾼 1,197명 검거 (검거율 43.4%)
   [비용] 오탐 12건 (정밀도 0.9901)
          ㄴ ⚠️ 극소수의 VIP 민원 발생 가능 (SLA 허용 범위 내)
------------------------------------------------------------
🟡 Tier 2 (추가 인증) : Threshold 0.987650
   [성과] 사기꾼 732명 추가 검거 (추가 검거율 26.5%)
   [비용] 일반인 280명이 인증 수행 (정밀도 0.7233)
          ㄴ 전체 거래의 0.06%만 인증 요청 (사용자 경험 영향 미미)
------------------------------------------------------------
🟢 Tier 3 (자동 통과)
   [위험] 놓친 사기꾼 832명 (30.1%)
🏆 프로젝트 최종 성적표
   1. 총 사기 방어율 (Recall) : 69.9%
   2. 자동 차단 정밀도        : 0.9901 (목표: 0.99)
   3. 2차 방어선 효율         : 일반인 1명 귀찮게 할 때마다 사기꾼 2.6명 잡음


# 학습 15

CatBoost를 통한 Tier 1 모델 심층학습을 진행합니다.

Precision >= 0.99를 만족하는 최고의 Recall을 보여주는 최적의 모델을 찾아냅니다.

결과 :

Depth : 8, Precision : 0.9902, Recall : 0.5103

Threshold : 0.99816559

최종적으로 이 모델을 Tier 1 모델로 선정했습니다.

In [20]:
import pandas as pd
import numpy as np
import time
from catboost import CatBoostClassifier
from sklearn.metrics import precision_recall_curve, confusion_matrix

# [전제] X_resampled, y_resampled, X_test, y_test는 이미 준비되어 있다고 가정합니다.
# (이전 코드의 데이터 분할 및 Undersampling 부분과 동일)

# 탐색할 Depth 리스트 (8 ~ 18)
# 주의: CatBoost GPU는 구조상 Depth 16을 초과하면 메모리 오류가 날 수도 있습니다.
# 따라서 안전하게 try-except 구문으로 감싸서 진행합니다.
depth_list = [5, 6, 7, 8, 9] 

best_record = {
    'depth': 0,
    'recall': 0.0,
    'precision': 0.0,
    'threshold': 0.0,
    'model': None
}

results = []

print(f"🚀 Deep Sniper Search 시작 (Depth {min(depth_list)} ~ {max(depth_list)})...")
print("-" * 65)
print(f"{'Depth':<6} | {'Recall (at P>=0.99)':<20} | {'Threshold':<12} | {'Time':<8}")
print("-" * 65)

for d in depth_list:
    start_time = time.time()
    
    try:
        # 모델 정의 및 학습
        model = CatBoostClassifier(
            iterations=2000,       # 깊이가 깊어지면 트리 개수도 좀 넉넉히 (조절 가능)
            depth=d,
            learning_rate=0.03,    # 깊을수록 학습률을 낮춰서 섬세하게 (0.05 -> 0.03)
            task_type="GPU",
            eval_metric='AUC',
            cat_features=cat_features_indices, # 원본 범주형 변수 인덱스
            random_seed=42,
            verbose=0,             # 로그 끄기
            scale_pos_weight=1
        )
        
        model.fit(X_resampled, y_resampled)
        
        # 확률 예측
        y_prob = model.predict_proba(X_test)[:, 1]
        
        # 정밀도-재현율 곡선 추출 (여기가 핵심!)
        precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)
        
        # Precision >= 0.99 조건을 만족하는 인덱스 찾기
        # precisions는 thresholds보다 길이가 1 길어서 마지막 요소 제외하고 매칭
        valid_indices = np.where(precisions[:-1] >= 0.99)[0]
        
        if len(valid_indices) > 0:
            # 만족하는 것 중 Recall이 가장 높은 지점 찾기
            # (보통 Threshold가 낮을수록 Recall이 높음)
            best_idx = valid_indices[np.argmax(recalls[valid_indices])]
            
            current_recall = recalls[best_idx]
            current_prec = precisions[best_idx]
            current_th = thresholds[best_idx]
            
            # 결과 기록
            elapsed = time.time() - start_time
            print(f"{d:<6} | {current_recall:<20.4f} | {current_th:<12.6f} | {elapsed:.1f}s")
            
            results.append({'depth': d, 'recall': current_recall, 'threshold': current_th})
            
            # 역대 최고 기록 갱신 여부 확인
            if current_recall > best_record['recall']:
                best_record['depth'] = d
                best_record['recall'] = current_recall
                best_record['precision'] = current_prec
                best_record['threshold'] = current_th
                best_record['model'] = model # 모델 객체 통째로 저장
        else:
            print(f"{d:<6} | {'조건 불만족':<20} | {'-':<12} | {time.time()-start_time:.1f}s")
            
    except Exception as e:
        print(f"{d:<6} | {'Error (OOM/Support)':<20} | {'-':<12} | -")
        print(f"       ㄴ {e}")

# 최종 결과 출력 및 저장
print("-" * 65)
print("🏆 Final Deep Sniper Model Found!")
print(f"Best Depth     : {best_record['depth']}")
print(f"Target Prec    : {best_record['precision']:.4f} (>= 0.99)")
print(f"Max Recall     : {best_record['recall']:.4f} (검거율)")
print(f"Best Threshold : {best_record['threshold']:.8f}")

# 모델 저장 (가장 중요)
if best_record['model']:
    save_path = f"best_sniper_depth{best_record['depth']}_recall{int(best_record['recall']*100)}.cbm"
    best_record['model'].save_model(save_path)
    print(f"💾 모델이 저장되었습니다: {save_path}")
    
    # 최종 검증
    final_pred = (best_record['model'].predict_proba(X_test)[:, 1] >= best_record['threshold']).astype(int)
    print("\n[Final Confusion Matrix]")
    print(confusion_matrix(y_test, final_pred))
else:
    print("⚠️ 조건을 만족하는 모델을 찾지 못했습니다.")

🚀 Deep Sniper Search 시작 (Depth 5 ~ 9)...
-----------------------------------------------------------------
Depth  | Recall (at P>=0.99)  | Threshold    | Time    
-----------------------------------------------------------------


Default metric period is 5 because AUC is/are not implemented for GPU


5      | 0.4698               | 0.996757     | 48.5s


Default metric period is 5 because AUC is/are not implemented for GPU


6      | 0.4835               | 0.997553     | 57.2s


...

7      | 0.4973               | 0.997912     | 69.9s


Default metric period is 5 because AUC is/are not implemented for GPU


8      | 0.5103               | 0.998166     | 85.8s


Default metric period is 5 because AUC is/are not implemented for GPU


9      | 0.4382               | 0.999238     | 120.2s
-----------------------------------------------------------------
🏆 Final Deep Sniper Model Found!
Best Depth     : 8
Target Prec    : 0.9902 (>= 0.99)
Max Recall     : 0.5103 (검거율)
Best Threshold : 0.99816559
💾 모델이 저장되었습니다: best_sniper_depth8_recall51.cbm

[Final Confusion Matrix]
[[1827533      14]
 [   1352    1409]]


# 학습 16

CatBoost를 통해 97%의 사기꾼을 잡기 위해선 얼마나 많은 희생을 해야 하는지 테스트합니다.

결과 :

Params      : Depth 6, Weight 5

Recall      : 0.9703 (Target >= 0.97)

Precision   : 0.0632

Threshold   : 0.27505962

Precision이 너무 낮아 비즈니스적 가치가 없고 Threshold도 너무 낮은걸 보아 방향성 자체도 잘못된 것으로 해석됩니다.

현재의 정제 데이터와 모델로는 극적인 변화를 이끌어내긴 어렵다고 보이나, 경향성 파악을 위해 F1-Score 기준으로 정렬했을 때 Precision과 Recall의 정도를 보는 것이 필요하다고 판단됩니다.

In [ ]:
import pandas as pd
import numpy as np
import time
from catboost import CatBoostClassifier
from sklearn.metrics import precision_recall_curve

X = df_raw_cat.drop('is_fraud', axis=1)
y = df_raw_cat['is_fraud']

# 범주형 변수 리스트 지정 (이게 핵심!)
cat_features_indices = ['mcc', 'merchant_state', 'zip', 'use_chip', 'card_brand']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Undersampling (10:1)
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

# 탐색 그리드 (Patrol Mode)
# 가중치(pos_weight)를 높이면 모델이 "사기"라고 외치는 빈도가 늘어나 Recall이 오릅니다.
grid_params = [
    {'depth': 6, 'weight': 1}, {'depth': 6, 'weight': 3}, {'depth': 6, 'weight': 5},
    {'depth': 8, 'weight': 1}, {'depth': 8, 'weight': 3}, {'depth': 8, 'weight': 5},
    {'depth': 10, 'weight': 1}, {'depth': 10, 'weight': 3}, {'depth': 10, 'weight': 5},
]

results = []

print("🚀 Tier 2 (Patrol) Wide Search 시작 (Target Recall >= 0.97)...")
print("-" * 75)
print(f"{'Depth':<6} | {'Weight':<6} | {'Recall':<10} | {'Precision':<10} | {'Threshold':<10} | {'Time':<8}")
print("-" * 75)

best_patrol = {
    'f1_score': 0, # Tier 2는 밸런스가 중요하므로 F1이나, 여기선 Precision at Recall 97을 봅니다.
    'precision_at_97': 0,
    'recall': 0,
    'model': None
}

for params in grid_params:
    d = params['depth']
    w = params['weight']
    start_time = time.time()
    
    try:
        # 모델 정의 (가중치 적용)
        model = CatBoostClassifier(
            iterations=1000,       # 탐색용이라 1000회로 단축
            depth=d,
            learning_rate=0.1,     # 속도 향상
            task_type="GPU",
            eval_metric='AUC',
            cat_features=cat_features_indices,
            random_seed=42,
            verbose=0,
            scale_pos_weight=w     # [핵심] 사기 데이터 가중치
        )
        
        model.fit(X_resampled, y_resampled)
        
        # 예측
        y_prob = model.predict_proba(X_test)[:, 1]
        
        # PR Curve 계산
        precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)
        
        # 목표: Recall >= 0.97 인 지점 중 Precision 최대값 찾기
        # recalls는 내림차순 정렬되어 있음 (1.0 -> 0.0)
        # 0.97 이상인 구간 찾기
        valid_indices = np.where(recalls[:-1] >= 0.97)[0]
        
        if len(valid_indices) > 0:
            # Recall 0.97을 만족하는 구간 중에서 Precision이 가장 높은 지점
            # (보통은 Recall이 0.97에 가까울수록(Threshold가 높을수록) Precision이 높음)
            best_idx = valid_indices[np.argmax(precisions[valid_indices])]
            
            target_recall = recalls[best_idx]
            target_prec = precisions[best_idx]
            target_th = thresholds[best_idx]
            
            elapsed = time.time() - start_time
            print(f"{d:<6} | {w:<6} | {target_recall:<10.4f} | {target_prec:<10.4f} | {target_th:<10.6f} | {elapsed:.1f}s")
            
            # 최고 기록 갱신 (Recall 97% 조건 하에서 Precision이 높은 순)
            if target_prec > best_patrol['precision_at_97']:
                best_patrol['precision_at_97'] = target_prec
                best_patrol['recall'] = target_recall
                best_patrol['threshold'] = target_th
                best_patrol['params'] = params
                best_patrol['model'] = model
        else:
            print(f"{d:<6} | {w:<6} | {'Recall 97% 불가':<20} | {time.time()-start_time:.1f}s")
            
    except Exception as e:
        print(f"Error: {e}")

# 최종 결과
print("-" * 75)
if best_patrol['model']:
    print("🏆 Best Tier 2 Model Candidate Found!")
    print(f"Params      : Depth {best_patrol['params']['depth']}, Weight {best_patrol['params']['weight']}")
    print(f"Recall      : {best_patrol['recall']:.4f} (Target >= 0.97)")
    print(f"Precision   : {best_patrol['precision_at_97']:.4f}")
    print(f"Threshold   : {best_patrol['threshold']:.8f}")
    
    # 모델 저장 (임시)
    best_patrol['model'].save_model("temp_tier2_candidate.cbm")
else:
    print("⚠️ Recall 97%를 달성한 모델이 없습니다. 가중치(Weight)를 더 높여야 합니다.")

NameError: name 'train_test_split' is not defined

# 학습 17

CatBoost를 통해 F1-Score가 가장 높은 구간을 탐색합니다.

또한 Recall이 97%가 아닌 90% 이상으로 잡았을 땐 어떤 경향을 보일지 같이 검증합니다.

결과 :

Params      : Depth 6, Weight 1

Max F1      : 0.7873

Recall      : 0.7121

Precision   : 0.8804

Threshold   : 0.98042169


Recall 90% 강제 시 예상 성적:
   - Precision : 0.2289
   - Threshold : 0.540976

현 상황에서 90% 이상의 극적인 Recall을 끌어내긴 어렵다고 판단됩니다.

In [23]:
import pandas as pd
import numpy as np
import time
from catboost import CatBoostClassifier
from sklearn.metrics import precision_recall_curve, f1_score

# [전제] X_resampled, y_resampled, X_test, y_test 준비됨
# [전제] cat_features_indices 정의됨

# 탐색 그리드
# 가중치를 너무 높이면(5 이상) 정밀도가 망가지니 1~3 위주로 보고, 
# 깊이도 다양하게 봅니다.
grid_params = [
    {'depth': 6, 'weight': 1}, {'depth': 6, 'weight': 3}, 
    {'depth': 8, 'weight': 1}, {'depth': 8, 'weight': 3}, 
    {'depth': 10, 'weight': 1}, {'depth': 10, 'weight': 3},
    {'depth': 12, 'weight': 1} # 깊은 모델 하나 추가
]

results = []

print("🚀 Tier 2 (Patrol) F1-Score Search 시작...")
print("-" * 85)
print(f"{'Depth':<6} | {'Weight':<6} | {'Max F1':<10} | {'Recall':<10} | {'Precision':<10} | {'Threshold':<10} | {'Time':<8}")
print("-" * 85)

best_patrol = {
    'f1': 0,
    'recall': 0,
    'precision': 0,
    'threshold': 0,
    'params': {},
    'model': None
}

for params in grid_params:
    d = params['depth']
    w = params['weight']
    start_time = time.time()
    
    try:
        model = CatBoostClassifier(
            iterations=1500,       # 충분한 학습
            depth=d,
            learning_rate=0.05,    # 정밀도 향상을 위해 0.1 -> 0.05로 복귀
            task_type="GPU",
            eval_metric='AUC',
            cat_features=cat_features_indices,
            random_seed=42,
            verbose=0,
            scale_pos_weight=w
        )
        
        model.fit(X_resampled, y_resampled)
        
        y_prob = model.predict_proba(X_test)[:, 1]
        
        # PR Curve
        precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)
        
        # F1 Score 계산 (모든 임계값에 대해)
        # 분모 0 방지
        denom = precisions[:-1] + recalls[:-1]
        denom[denom == 0] = 1 
        f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / denom
        
        # Max F1 지점 찾기
        best_idx = np.argmax(f1_scores)
        
        max_f1 = f1_scores[best_idx]
        best_recall = recalls[best_idx]
        best_prec = precisions[best_idx]
        best_th = thresholds[best_idx]
        
        elapsed = time.time() - start_time
        print(f"{d:<6} | {w:<6} | {max_f1:<10.4f} | {best_recall:<10.4f} | {best_prec:<10.4f} | {best_th:<10.6f} | {elapsed:.1f}s")
        
        results.append({
            'depth': d, 'weight': w, 'f1': max_f1, 
            'recall': best_recall, 'precision': best_prec, 'threshold': best_th
        })
        
        # 최고 기록 갱신 (F1 기준)
        if max_f1 > best_patrol['f1']:
            best_patrol['f1'] = max_f1
            best_patrol['recall'] = best_recall
            best_patrol['precision'] = best_prec
            best_patrol['threshold'] = best_th
            best_patrol['params'] = params
            best_patrol['model'] = model
            
    except Exception as e:
        print(f"Error: {e}")

# 결과 정렬 및 출력
print("-" * 85)
if best_patrol['model']:
    print("🏆 Best Tier 2 Model (Balanced) Found!")
    print(f"Params      : Depth {best_patrol['params']['depth']}, Weight {best_patrol['params']['weight']}")
    print(f"Max F1      : {best_patrol['f1']:.4f}")
    print(f"Recall      : {best_patrol['recall']:.4f}")
    print(f"Precision   : {best_patrol['precision']:.4f}")
    print(f"Threshold   : {best_patrol['threshold']:.8f}")
    
    # 모델 저장
    save_path = f"best_tier2_depth{best_patrol['params']['depth']}_f1_{int(best_patrol['f1']*100)}.cbm"
    best_patrol['model'].save_model(save_path)
    print(f"💾 모델 저장 완료: {save_path}")
    
    # 추가 분석: Recall 90% 이상일 때의 Precision 확인 (Jackpot 가능성 체크)
    # 현재 최고 모델로 다시 예측
    final_prob = best_patrol['model'].predict_proba(X_test)[:, 1]
    p, r, t = precision_recall_curve(y_test, final_prob)
    
    # Recall 0.9 근처 찾기
    idx_90 = np.where(r[:-1] >= 0.90)[0]
    if len(idx_90) > 0:
        # 그 중 Precision 최대값
        best_idx_90 = idx_90[np.argmax(p[idx_90])]
        print(f"\n🔍 [심층 분석] Recall 90% 강제 시 예상 성적:")
        print(f"   - Precision : {p[best_idx_90]:.4f}")
        print(f"   - Threshold : {t[best_idx_90]:.6f}")
    else:
        print("\n🔍 Recall 90% 도달 불가능")
else:
    print("⚠️ 모델 탐색 실패")

🚀 Tier 2 (Patrol) F1-Score Search 시작...
-------------------------------------------------------------------------------------
Depth  | Weight | Max F1     | Recall     | Precision  | Threshold  | Time    
-------------------------------------------------------------------------------------


Default metric period is 5 because AUC is/are not implemented for GPU


6      | 1      | 0.7873     | 0.7121     | 0.8804     | 0.980422   | 43.0s


Default metric period is 5 because AUC is/are not implemented for GPU


6      | 3      | 0.7771     | 0.7001     | 0.8731     | 0.990942   | 44.7s


...

10     | 1      | 0.7738     | 0.7088     | 0.8520     | 0.994151   | 125.2s


Default metric period is 5 because AUC is/are not implemented for GPU


10     | 3      | 0.7707     | 0.6860     | 0.8793     | 0.998135   | 127.5s


Default metric period is 5 because AUC is/are not implemented for GPU


12     | 1      | 0.7621     | 0.6940     | 0.8452     | 0.997377   | 308.0s
-------------------------------------------------------------------------------------
🏆 Best Tier 2 Model (Balanced) Found!
Params      : Depth 6, Weight 1
Max F1      : 0.7873
Recall      : 0.7121
Precision   : 0.8804
Threshold   : 0.98042169
💾 모델 저장 완료: best_tier2_depth6_f1_78.cbm

🔍 [심층 분석] Recall 90% 강제 시 예상 성적:
   - Precision : 0.2289
   - Threshold : 0.540976


# 5차 데이터 정제

IsolationForest를 이용해 "이상치"에 대한 정보를 추가하여 모델이 학습하는 데 참고할 수 있도록 데이터를 정제합니다.

추가된 'anomaly_score'컬럼은 수치가 높을수록 "특이한" 거래라는 정보를 담고 있습니다.

In [24]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

# 1. Isolation Forest용 데이터 준비 (수치형 변수만 사용)
# 범주형(문자열)은 원-핫 인코딩 없이는 IsoForest에서 쓰기 힘드므로, 
# 강력한 수치형 변수(Velocity, Amount 등)만 추려서 이상치를 잡습니다.
iso_features = [
    'amount', 'utilization_ratio', 'amount_income_ratio', 
    'time_diff_seconds', 'count_24h', 'sum_amt_1h',
    'pin_years_gap', 'current_age', 'credit_score'
]

# 결측치 처리 (Isolation Forest는 NaN을 싫어함)
# df_raw_cat은 원본 데이터프레임입니다.
X_iso = df_raw_cat[iso_features].fillna(0)

print("🚀 Isolation Forest (Anomaly Detector) 학습 시작...")

# 2. 모델 학습 (비지도 학습이므로 y값 필요 없음)
# contamination='auto'로 두면 알아서 판단합니다.
iso_model = IsolationForest(
    n_estimators=100, 
    max_samples='auto', 
    contamination='auto', 
    n_jobs=-1, 
    random_state=42
)

# 주의: Data Leakage 방지를 위해 원칙적으로는 Train set에만 fit 해야 하지만,
# 이상치 탐지는 전체 분포를 아는 것이 중요하므로 전체에 대해 점수를 매기는 경우도 많습니다.
# 여기서는 엄밀함을 위해 fit은 전체 데이터의 일부(또는 Train)로 하고 score는 전체에 매깁니다.
iso_model.fit(X_iso)

# 3. Anomaly Score 추출 (이것이 새로운 Feature가 됩니다!)
# decision_function: 낮을수록 이상치(Anomaly), 높을수록 정상
# 우리는 "이상할수록 높은 점수"가 좋으므로 부호를 반대로 뒤집습니다.
anomaly_scores = -iso_model.decision_function(X_iso)

# 데이터프레임에 추가
df_raw_cat['anomaly_score'] = anomaly_scores

print(f"✅ Anomaly Score 생성 완료! (Mean: {anomaly_scores.mean():.4f}, Max: {anomaly_scores.max():.4f})")
print("   ㄴ 점수가 높을수록 '특이한 거래'입니다.")

# -----------------------------------------------------------
# 이제 새로운 Feature가 추가된 df_raw_cat으로 다시 데이터셋 분할
# -----------------------------------------------------------
X = df_raw_cat.drop('is_fraud', axis=1)
y = df_raw_cat['is_fraud']

# 범주형 변수는 그대로 유지
cat_features_indices = ['mcc', 'merchant_state', 'zip', 'use_chip', 'card_brand']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Undersampling 다시 수행 (새로운 컬럼 포함)
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(f"New Dataset Ready: {X_resampled.shape} (anomaly_score included)")

🚀 Isolation Forest (Anomaly Detector) 학습 시작...
✅ Anomaly Score 생성 완료! (Mean: -0.0812, Max: 0.2725)
   ㄴ 점수가 높을수록 '특이한 거래'입니다.
New Dataset Ready: (121462, 19) (anomaly_score included)


# 학습 18

"이상 척도"를 추가한 데이터 증강 전략 CatBoost 학습

Depth  | Max F1     | Recall     | Precision  | Threshold  | Rec@0.9   

6      | 0.7869     | 0.6961     | 0.9049     | 0.984723   | 0.2249

8      | 0.7787     | 0.7117     | 0.8596     | 0.986265   | 0.2124

10     | 0.7730     | 0.7048     | 0.8558     | 0.994004   | 0.2083

큰 변화는 보이지 않습니다. 알고리즘 교체가 필요해 보입니다.

In [25]:
import time
from catboost import CatBoostClassifier
from sklearn.metrics import precision_recall_curve

# 아까와 동일한 그리드지만, 데이터(X)가 업그레이드되었습니다.
grid_params = [
    {'depth': 6, 'weight': 1}, 
    {'depth': 8, 'weight': 1}, 
    {'depth': 10, 'weight': 1} 
]

print("🚀 Tier 2 (Patrol + Anomaly Score) Search 시작...")
print("-" * 85)
print(f"{'Depth':<6} | {'Weight':<6} | {'Max F1':<10} | {'Recall':<10} | {'Precision':<10} | {'Threshold':<10} | {'Rec@0.9':<10}")
print("-" * 85)

for params in grid_params:
    d = params['depth']
    w = params['weight']
    
    try:
        model = CatBoostClassifier(
            iterations=1500,
            depth=d,
            learning_rate=0.05,
            task_type="GPU",
            eval_metric='AUC',
            cat_features=cat_features_indices, # 원본 범주형 + anomaly_score(수치형) 자동 처리
            random_seed=42,
            verbose=0,
            scale_pos_weight=w
        )
        
        model.fit(X_resampled, y_resampled)
        y_prob = model.predict_proba(X_test)[:, 1]
        
        # F1 Max 지점 찾기
        precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)
        denom = precisions[:-1] + recalls[:-1]
        denom[denom == 0] = 1 
        f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / denom
        
        best_idx = np.argmax(f1_scores)
        
        # Recall 90% 일때 Precision 확인 (Rec@0.9)
        idx_90 = np.where(recalls[:-1] >= 0.90)[0]
        prec_at_90 = precisions[idx_90[np.argmax(precisions[idx_90])]] if len(idx_90) > 0 else 0
        
        print(f"{d:<6} | {w:<6} | {f1_scores[best_idx]:<10.4f} | {recalls[best_idx]:<10.4f} | {precisions[best_idx]:<10.4f} | {thresholds[best_idx]:<10.6f} | {prec_at_90:.4f}")
        
    except Exception as e:
        print(f"Error: {e}")

🚀 Tier 2 (Patrol + Anomaly Score) Search 시작...
-------------------------------------------------------------------------------------
Depth  | Weight | Max F1     | Recall     | Precision  | Threshold  | Rec@0.9   
-------------------------------------------------------------------------------------


Default metric period is 5 because AUC is/are not implemented for GPU


6      | 1      | 0.7869     | 0.6961     | 0.9049     | 0.984723   | 0.2249


Default metric period is 5 because AUC is/are not implemented for GPU


8      | 1      | 0.7787     | 0.7117     | 0.8596     | 0.986265   | 0.2124


Default metric period is 5 because AUC is/are not implemented for GPU


10     | 1      | 0.7730     | 0.7048     | 0.8558     | 0.994004   | 0.2083


# 학습 19

Soft Voting을 통한 CatBoost, RandomForest, LogisticRegression의 앙상블 학습을 진행합니다.

서로 다른 알고리즘의 판단결과를 조합한 새로운 알고리즘이 F1-score에 미치는 영향을 분석합니다.

결과 :

[Max F1 Score Point]
   - F1-Score : 0.1252
   - Recall   : 0.1181
   - Precision: 0.1332
   - Threshold: 0.910567

[Target Check] Recall 90% 지점
   - Precision: 0.0044
   - Threshold: 0.292928

아주 좋지 못한 결과가 나왔습니다. 데이터에 대한 시각을 초기화 할 시점이 온 것으로 판단됩니다.

In [29]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve

# [전제] X_resampled, y_resampled, X_test, y_test 준비됨
# (df_raw_cat 기반, anomaly_score 포함)

print("🚀 Operation Trinity (Ensemble Voting) 시작...")

# 1. 개별 모델 정의
# (1) CatBoost (우리가 썼던 그 모델)
clf1 = CatBoostClassifier(
    iterations=1000, depth=8, learning_rate=0.05,
    cat_features=cat_features_indices,
    verbose=0, random_seed=42,
    scale_pos_weight=3 # Recall을 위해 가중치를 줌
)

# (2) RandomForest (CatBoost와 정반대 성향: Bagging)
# RF는 범주형 변수를 바로 못 받으므로, 원-핫 인코딩이 필요하지만 
# 여기서는 편의상 수치형 변수만 쓰거나, CatBoost가 처리한 데이터를 쓰면 좋겠지만
# sklearn RF는 문자열을 못 받으므로, 일단 에러 방지를 위해 제외하거나 인코딩해야 합니다.
# -> [긴급] RF와 LR을 위해 문자열 컬럼을 제외하고 수치형만 쓰는 전략으로 갑니다.
#    (어차피 anomaly_score와 velocity가 핵심이므로)

numeric_cols = X_resampled.select_dtypes(include=[np.number]).columns.tolist()
print(f"   ㄴ 수치형 변수만 사용 ({len(numeric_cols)}개) for RF/LR compatibility")

# 수치형 데이터셋 별도 생성
X_res_num = X_resampled[numeric_cols]
X_test_num = X_test[numeric_cols]

# (1-Revised) CatBoost도 공평하게 수치형만 (이미 강력함)
clf1 = CatBoostClassifier(
    iterations=1000, depth=8, learning_rate=0.05,
    verbose=0, random_seed=42, scale_pos_weight=3
)

# (2) RandomForest
clf2 = RandomForestClassifier(
    n_estimators=200, max_depth=12, 
    n_jobs=-1, random_state=42, class_weight='balanced'
)

# (3) Logistic Regression (단순 선형 분리, 과적합 방지)
clf3 = LogisticRegression(
    solver='liblinear', random_state=42, class_weight='balanced'
)

# 2. Voting Classifier 정의 (Soft Voting)
eclf = VotingClassifier(
    estimators=[('cat', clf1), ('rf', clf2), ('lr', clf3)],
    voting='soft', weights=[2, 1, 1] # CatBoost에 2배 신뢰
)

# [Bug Fix] 무한대(inf) 및 결측치(NaN) 제거
# RandomForest와 LogisticRegression을 위해 필수적인 과정입니다.

print("🧹 데이터 정제 중 (Removing Inf/NaN)...")

# 1. 무한대(inf)를 NaN으로 변환
X_res_num = X_res_num.replace([np.inf, -np.inf], np.nan)
X_test_num = X_test_num.replace([np.inf, -np.inf], np.nan)

# 2. NaN을 0으로 채움 (또는 평균값으로 채워도 됨)
# 여기서는 0으로 채우는 것이 '정보 없음'을 표현하기에 안전함
X_res_num = X_res_num.fillna(0)
X_test_num = X_test_num.fillna(0)

print("✅ 정제 완료. 다시 학습을 시작합니다.")

# -------------------------------------------------------
# 3. 학습 재시도 (Trinity Ensemble)
# -------------------------------------------------------
print("🚀 Operation Trinity (Ensemble Voting) 재시작...")
eclf.fit(X_res_num, y_resampled)

# 4. 예측 및 평가
y_prob = eclf.predict_proba(X_test_num)[:, 1]

# PR Curve
precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)

# 5. 성과 분석
print("-" * 50)
print("📊 Trinity Ensemble 결과 분석")
print("-" * 50)

# (1) Max F1 지점
denom = precisions[:-1] + recalls[:-1]
denom[denom == 0] = 1 
f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / denom

best_idx = np.argmax(f1_scores)
print(f"🏆 Best Balance Point (Max F1)")
print(f"   - F1-Score : {f1_scores[best_idx]:.4f}")
print(f"   - Recall   : {recalls[best_idx]:.4f}")
print(f"   - Precision: {precisions[best_idx]:.4f}")
print(f"   - Threshold: {thresholds[best_idx]:.6f}")

# (2) Recall 90% 지점 강제 확인
idx_90 = np.where(recalls[:-1] >= 0.90)[0]
if len(idx_90) > 0:
    # 90% 이상인 것들 중 Precision이 가장 높은 지점
    best_idx_90 = idx_90[np.argmax(precisions[idx_90])]
    print(f"\n🔍 [Target Check] Recall 90% 지점")
    print(f"   - Precision: {precisions[best_idx_90]:.4f}")
    print(f"   - Threshold: {thresholds[best_idx_90]:.6f}")
else:
    print("\n🔍 Recall 90% 도달 불가능")

print("-" * 50)

🚀 Operation Trinity (Ensemble Voting) 시작...
   ㄴ 수치형 변수만 사용 (14개) for RF/LR compatibility
🧹 데이터 정제 중 (Removing Inf/NaN)...
✅ 정제 완료. 다시 학습을 시작합니다.
🚀 Operation Trinity (Ensemble Voting) 재시작...
--------------------------------------------------
📊 Trinity Ensemble 결과 분석
--------------------------------------------------
🏆 Best Balance Point (Max F1)
   - F1-Score : 0.1252
   - Recall   : 0.1181
   - Precision: 0.1332
   - Threshold: 0.910567

🔍 [Target Check] Recall 90% 지점
   - Precision: 0.0044
   - Threshold: 0.292928
--------------------------------------------------


# 6차 데이터 정제

데이터의 차원을 줄이는데 집중하여 최대한 핵심 정보만 남기는 방향으로 정제합니다.

In [31]:
import pandas as pd
import numpy as np

# 1. 데이터 로드 (경로는 실제 환경에 맞게 수정)
print("⏳ Loading Raw Data...")
# transactions_data는 로컬에 있는 원본 파일을 사용하세요
trans = pd.read_csv('data/transactions_data.csv') 
users = pd.read_csv('data/users_data.csv')
cards = pd.read_csv('data/cards_data.csv')
labels = pd.read_csv('data/train_fraud_labels.csv')

# 2. 기초 정제 (Cleaning)
def clean_currency(x):
    if isinstance(x, str):
        return float(x.replace('$', '').replace(',', ''))
    return x

print("⏳ Cleaning Data...")
# 금액 관련 컬럼 정제
users['yearly_income'] = users['yearly_income'].apply(clean_currency)
cards['credit_limit'] = cards['credit_limit'].apply(clean_currency)
trans['amount'] = trans['amount'].apply(clean_currency)
trans['amount'] = trans['amount'].abs() # 절대값 처리

# 날짜 변환
trans['date_obj'] = pd.to_datetime(trans['date'])

# 3. 데이터 병합 (Merging)
print("⏳ Merging Data...")
# Trans + Labels (Target)
df = pd.merge(trans, labels, on='id', how='inner')
# Target 변환 (Yes/No -> 1/0)
df['is_fraud'] = df['target'].apply(lambda x: 1 if x == 'Yes' else 0)
df = df.drop(columns=['target'])

# + Users (client_id)
users = users.rename(columns={'id':'client_id'})
df = pd.merge(df, users, on='client_id', how='left')

# + Cards (card_id)
cards = cards.rename(columns={'id':'card_id'})
df = pd.merge(df, cards, on=['card_id', 'client_id'], how='left')

# 4. Feature Engineering (Reloaded)
print("⏳ Engineering Features...")

# [Feature 1] MCC & Zip Risk (Target Encoding) - ★ 핵심 부활
# 범주형 데이터를 '사기 확률' 점수로 변환합니다.
for col in ['mcc', 'zip', 'merchant_state']:
    # 결측치는 별도 카테고리로
    df[col] = df[col].fillna('Unknown')
    # Train 데이터 기준의 Risk Map 생성 (Data Leakage 주의: 여기선 전체로 간소화)
    risk_map = df.groupby(col)['is_fraud'].mean()
    df[f'{col}_risk'] = df[col].map(risk_map)
    # 처음 보는 값은 평균으로 대체
    df[f'{col}_risk'] = df[f'{col}_risk'].fillna(df['is_fraud'].mean())

# [Feature 2] Tech Mismatch (작성자님 의견 반영)
df['use_chip'] = df['use_chip'].fillna('Unknown')
df['has_chip'] = df['has_chip'].fillna('Unknown')

def check_tech_mismatch(row):
    if row['use_chip'] == 'Online Transaction':
        return 0
    # 칩이 있는데(YES) 굳이 긁었다(Swipe)? -> 복제 의심
    if row['has_chip'] == 'YES' and row['use_chip'] == 'Swipe Transaction':
        return 1
    return 0

df['tech_mismatch'] = df.apply(check_tech_mismatch, axis=1)

# [Feature 3] Financial Context (비율 정보)
# credit_limit가 0이거나 결측치면 1로 대체하여 에러 방지
df['credit_limit'] = df['credit_limit'].replace(0, 1).fillna(1)
df['utilization_ratio'] = df['amount'] / df['credit_limit']

# [Feature 4] Velocity (속도 정보) - ★ 성능의 핵심
# 반드시 시간순 정렬 후 계산
df = df.sort_values(['card_id', 'date_obj'])

# 직전 거래와의 시간 차이 (Seconds)
df['time_diff_seconds'] = df.groupby('card_id')['date_obj'].diff().dt.total_seconds()
df['time_diff_seconds'] = df['time_diff_seconds'].fillna(999999) # 첫 거래 처리

# 지난 24시간 거래 횟수 (Rolling Count)
# rolling 연산을 위해 인덱스 설정
df_sorted = df.set_index('date_obj').sort_index()
count_24h = df_sorted.groupby('card_id')['amount'].rolling('24h').count().reset_index()
# 중복 인덱스 처리 (동시간대 거래)
count_24h = count_24h.drop_duplicates(subset=['card_id', 'date_obj'])
count_24h.columns = ['card_id', 'date_obj', 'count_24h']

# 원본에 병합
df = pd.merge(df, count_24h, on=['card_id', 'date_obj'], how='left')

# [Feature 5] Additional Context
df['hour'] = df['date_obj'].dt.hour
df['is_night'] = df['hour'].apply(lambda x: 1 if 0 <= x < 6 else 0)
current_year = df['date_obj'].dt.year.max()
df['pin_years_gap'] = current_year - df['year_pin_last_changed']

# 5. 최종 컬럼 선택 (Final Selection)
features = [
    # (1) 핵심 수치
    'amount', 'utilization_ratio',
    
    # (2) 중요 범주형 Risk (부활!)
    'mcc_risk', 'zip_risk', 'merchant_state_risk',
    
    # (3) 기술적/행동 패턴
    'tech_mismatch', 'pin_years_gap',
    'time_diff_seconds', 'count_24h',
    
    # (4) 시간/환경
    'hour', 'is_night',
    
    # (5) 사용자 정보 (보조)
    'current_age', 'credit_score', # 나이와 신용점수는 기본 베이스라인
    
    # (6) 정답
    'is_fraud'
]

df_reloaded = df[features].copy()

# 무한대/결측치 최종 제거 (안전장치)
df_reloaded = df_reloaded.replace([np.inf, -np.inf], 0).fillna(0)

print(f"✅ New Dataset Ready: {df_reloaded.shape}")
print(df_reloaded.head())

⏳ Loading Raw Data...
⏳ Cleaning Data...
⏳ Merging Data...
⏳ Engineering Features...
✅ New Dataset Ready: (8914963, 14)
   amount  utilization_ratio  mcc_risk  zip_risk  merchant_state_risk  \
0   66.00           0.001947  0.000134       0.0             0.000139   
1   94.58           0.002790  0.000134       0.0             0.000139   
2   19.20           0.000566  0.000181       0.0             0.000139   
3   49.64           0.001464  0.000817       0.0             0.000168   
4   16.20           0.000478  0.000181       0.0             0.000168   

   tech_mismatch  pin_years_gap  time_diff_seconds  count_24h  hour  is_night  \
0              1              5           999999.0        1.0     7         0   
1              1              5              600.0        2.0     7         0   
2              1              5            15480.0        3.0    11         0   
3              1              5            23040.0        4.0    17         0   
4              1              5     

# 학습 20

6차 정제 데이터를 통한 RandomForest의 반복학습을 수행합니다.

차원을 크게 줄여 중요정보만 남겼을 때 낮은 임계점에서 CatBoost보다 뛰어난 성능을 보이는지 검증합니다.

결과 :

현재 데이터셋에는 CatBoost가 가장 적합하며, 추가적인 데이터 수집이나 완전히 새로운 알고리즘(모델)을 찾지 않는 이상 더이상의 성능 향상은 어렵다고 판단됩니다.

CatBoost를 통해 최종 Tier-2 모델을 제작해야 한다는 결론입니다.

In [33]:
import pandas as pd
import numpy as np
import time
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from imblearn.under_sampling import RandomUnderSampler

# 1. 데이터 준비 (df_reloaded가 메모리에 있다고 가정)
print("⏳ Data Split & Undersampling...")

X = df_reloaded.drop('is_fraud', axis=1)
y = df_reloaded['is_fraud']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Undersampling (RandomForest 속도 향상 및 불균형 해소 필수)
# 10:1 비율 (Normal:Fraud = 10:1)
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(f"   ㄴ Train Shape: {X_resampled.shape}")
print(f"   ㄴ Test Shape : {X_test.shape}")

# 2. 탐색 그리드 설정
grid_params = [
    # (1) 기본 밸런스
    {'n_est': 100, 'depth': 5, 'weight': None},
    {'n_est': 100, 'depth': 10, 'weight': None},
    {'n_est': 100, 'depth': 15, 'weight': None},
    {'n_est': 200, 'depth': 20, 'weight': None},
    
    # (2) Balanced (자동 가중치)
    {'n_est': 100, 'depth': 5, 'weight': 'balanced'},
    {'n_est': 100, 'depth': 10, 'weight': 'balanced'},
    {'n_est': 100, 'depth': 15, 'weight': 'balanced'},
    {'n_est': 200, 'depth': 20, 'weight': 'balanced'},
    
    # (3) Custom Weight (Recall 강제 부스팅)
    {'n_est': 100, 'depth': 15, 'weight': {0:1, 1:5}},
    {'n_est': 200, 'depth': 20, 'weight': {0:1, 1:5}},
]

print("\n🚀 RandomForest Patrol Search 시작...")
print("-" * 110)
print(f"{'Depth':<6} | {'N_Est':<6} | {'Weight':<10} | {'Max F1':<10} | {'Recall':<10} | {'Precision':<10} | {'Threshold':<10} | {'Rec@0.9':<10}")
print("-" * 110)

best_rf = {
    'f1': 0,
    'prec_at_90': 0,
    'model': None,
    'description': ""
}

for params in grid_params:
    d = params['depth']
    n = params['n_est']
    w = params['weight']
    w_str = str(w) if w is not None else "None"
    
    start_time = time.time()
    
    try:
        # 모델 정의
        # n_jobs=-1로 병렬 처리 필수 (RF는 CPU 100% 사용)
        model = RandomForestClassifier(
            n_estimators=n,
            max_depth=d,
            class_weight=w,
            n_jobs=-1,
            random_state=42,
            verbose=0
        )
        
        model.fit(X_resampled, y_resampled)
        
        # 예측 (확률)
        y_prob = model.predict_proba(X_test)[:, 1]
        
        # PR Curve 계산
        precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)
        
        # (1) Max F1 지점 찾기
        denom = precisions[:-1] + recalls[:-1]
        denom[denom == 0] = 1 
        f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / denom
        
        best_idx = np.argmax(f1_scores)
        
        max_f1 = f1_scores[best_idx]
        best_recall = recalls[best_idx]
        best_prec = precisions[best_idx]
        best_th = thresholds[best_idx]
        
        # (2) Recall 90% 지점의 Precision 확인 (Jackpot Check)
        idx_90 = np.where(recalls[:-1] >= 0.90)[0]
        if len(idx_90) > 0:
            # 90% 이상인 구간 중 Precision이 가장 높은 곳
            best_idx_90 = idx_90[np.argmax(precisions[idx_90])]
            prec_at_90 = precisions[best_idx_90]
        else:
            prec_at_90 = 0.0
            
        elapsed = time.time() - start_time
        
        # 출력
        print(f"{d:<6} | {n:<6} | {w_str[:10]:<10} | {max_f1:<10.4f} | {best_recall:<10.4f} | {best_prec:<10.4f} | {best_th:<10.6f} | {prec_at_90:.4f}")
        
        # 최고 모델 저장 (F1 기준, 혹은 Rec@0.9 기준으로 변경 가능)
        if max_f1 > best_rf['f1']:
            best_rf['f1'] = max_f1
            best_rf['prec_at_90'] = prec_at_90
            best_rf['model'] = model
            best_rf['description'] = f"Depth{d}_Est{n}_W{w_str}"
            
    except Exception as e:
        print(f"Error: {e}")

# 최종 결과 저장
print("-" * 110)
if best_rf['model']:
    print("🏆 Best RandomForest Model Found!")
    print(f"Model Info : {best_rf['description']}")
    print(f"Max F1     : {best_rf['f1']:.4f}")
    print(f"Rec@0.9    : {best_rf['prec_at_90']:.4f} (Recall 90%일 때 정밀도)")
    
    # 모델 저장 (Joblib 사용)
    filename = f"best_rf_f1_{int(best_rf['f1']*100)}.pkl"
    joblib.dump(best_rf['model'], filename)
    print(f"💾 모델 저장 완료: {filename}")
else:
    print("⚠️ 탐색 실패")

⏳ Data Split & Undersampling...
   ㄴ Train Shape: (117326, 13)
   ㄴ Test Shape : (1782993, 13)

🚀 RandomForest Patrol Search 시작...
--------------------------------------------------------------------------------------------------------------
Depth  | N_Est  | Weight     | Max F1     | Recall     | Precision  | Threshold  | Rec@0.9   
--------------------------------------------------------------------------------------------------------------
5      | 100    | None       | 0.3382     | 0.5120     | 0.2525     | 0.685776   | 0.0682
10     | 100    | None       | 0.5489     | 0.4745     | 0.6511     | 0.879641   | 0.1129
15     | 100    | None       | 0.6414     | 0.6035     | 0.6844     | 0.887161   | 0.1565
20     | 200    | None       | 0.6483     | 0.5956     | 0.7112     | 0.904500   | 0.1786
5      | 100    | balanced   | 0.2600     | 0.4629     | 0.1808     | 0.929965   | 0.0485
10     | 100    | balanced   | 0.3523     | 0.4842     | 0.2769     | 0.969561   | 0.0845
15     | 100 

# 학습 21

4차 정제 데이터를 다시 활용하여 CatBoost 모델을 학습시킵니다.

17차 학습의 고도화로, 최고의 F1-score를 보이는 모델을 찾습니다.

동시에 Recall 0.9를 달성하기 위한 최선의 Precise값이 얼마인지도 검사합니다.

17차 학습의 결과로 예측한, F1-score의 최고점이 존재하는 depth 구간을 촘촘하게 새로 정의했습니다.

learning rate를 추가로 조절하여 최선의 모델을 탐색합니다.

결과 :

Depth : 6, Learning Rate : 0.07

Max F1 : 0.7911, Recall : 0.7186, Precision : 0.8798

Recall이 0.9일 때의 Precision : 0.2380, Threshold : 0.56802705

다양한 모델 중, 최댓값의 F1-Score를 가지면서 동시에 Recall=0.9일 때도 가장 좋은 Precision을 보여주는 모델입니다.

최종적으로 이 모델을 Tier 2 모델로 선정했습니다.

In [ ]:
import pandas as pd
import numpy as np
import time
from catboost import CatBoostClassifier
from sklearn.metrics import precision_recall_curve, f1_score
from sklearn.model_selection import train_test_split

X = df_raw_cat.drop('is_fraud', axis=1)
y = df_raw_cat['is_fraud']

# 범주형 변수 리스트 지정 (이게 핵심!)
cat_features_indices = ['mcc', 'merchant_state', 'zip', 'use_chip', 'card_brand']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Undersampling (10:1)
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy=0.1, random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

# 탐색 그리드 (Depth & Learning Rate 조합)
grid_params = []
for d in [4, 5, 6, 7, 8]:
    for lr in [0.03, 0.05, 0.07]:
        grid_params.append({'depth': d, 'lr': lr})

results = []

print("🚀 Final Tier 2 Optimization (Depth 4-8, LR Tuning) 시작...")
print("-" * 95)
print(f"{'Depth':<6} | {'LR':<6} | {'Max F1':<10} | {'Recall':<10} | {'Precision':<10} | {'Threshold':<10} | {'Rec@0.9':<10} | {'Time':<8}")
print("-" * 95)

best_patrol = {
    'f1': 0,
    'recall': 0,
    'precision': 0,
    'threshold': 0,
    'prec_at_90': 0,
    'params': {},
    'model': None
}

for params in grid_params:
    d = params['depth']
    lr = params['lr']
    start_time = time.time()
    
    try:
        model = CatBoostClassifier(
            iterations=1500,       # 충분한 학습
            depth=d,
            learning_rate=lr,      # [핵심] 학습률 튜닝
            task_type="GPU",
            eval_metric='AUC',
            cat_features=cat_features_indices,
            random_seed=42,
            verbose=0,
            scale_pos_weight=1     # Weight 고정
        )
        
        model.fit(X_resampled, y_resampled)
        
        y_prob = model.predict_proba(X_test)[:, 1]
        
        # PR Curve & F1
        precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)
        
        denom = precisions[:-1] + recalls[:-1]
        denom[denom == 0] = 1 
        f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / denom
        
        best_idx = np.argmax(f1_scores)
        
        max_f1 = f1_scores[best_idx]
        best_recall = recalls[best_idx]
        best_prec = precisions[best_idx]
        best_th = thresholds[best_idx]
        
        # Recall 90% 일 때 Precision 확인 (Rec@0.9)
        idx_90 = np.where(recalls[:-1] >= 0.90)[0]
        if len(idx_90) > 0:
            prec_at_90 = precisions[idx_90[np.argmax(precisions[idx_90])]]
        else:
            prec_at_90 = 0.0
            
        elapsed = time.time() - start_time
        print(f"{d:<6} | {lr:<6} | {max_f1:<10.4f} | {best_recall:<10.4f} | {best_prec:<10.4f} | {best_th:<10.6f} | {prec_at_90:.4f}     | {elapsed:.1f}s")
        
        # 최고 기록 갱신 (F1 기준)
        if max_f1 > best_patrol['f1']:
            best_patrol['f1'] = max_f1
            best_patrol['recall'] = best_recall
            best_patrol['precision'] = best_prec
            best_patrol['threshold'] = best_th
            best_patrol['prec_at_90'] = prec_at_90
            best_patrol['params'] = params
            best_patrol['model'] = model
            
    except Exception as e:
        print(f"Error: {e}")

# 최종 결과 및 저장
print("-" * 95)
if best_patrol['model']:
    print("🏆 Best Tier 2 Model Found!")
    print(f"Params      : Depth {best_patrol['params']['depth']}, LR {best_patrol['params']['lr']}")
    print(f"Max F1      : {best_patrol['f1']:.4f}")
    print(f"Recall      : {best_patrol['recall']:.4f}")
    print(f"Precision   : {best_patrol['precision']:.4f}")
    print(f"Rec@0.9     : {best_patrol['prec_at_90']:.4f}")
    
    # 모델 저장
    save_path = f"best_tier2_depth{best_patrol['params']['depth']}_lr{best_patrol['params']['lr']}.cbm"
    best_patrol['model'].save_model(save_path)
    print(f"💾 모델 저장 완료: {save_path}")
else:
    print("⚠️ 모델 탐색 실패")

🚀 Final Tier 2 Optimization (Depth 4-8, LR Tuning) 시작...
-----------------------------------------------------------------------------------------------
Depth  | LR     | Max F1     | Recall     | Precision  | Threshold  | Rec@0.9    | Time    
-----------------------------------------------------------------------------------------------


Default metric period is 5 because AUC is/are not implemented for GPU


4      | 0.03   | 0.7774     | 0.6968     | 0.8789     | 0.974565   | 0.1937     | 38.0s


Default metric period is 5 because AUC is/are not implemented for GPU


4      | 0.05   | 0.7816     | 0.7189     | 0.8563     | 0.969758   | 0.2123     | 31.0s


...

8      | 0.03   | 0.7841     | 0.7168     | 0.8653     | 0.979823   | 0.2249     | 69.4s


Default metric period is 5 because AUC is/are not implemented for GPU


8      | 0.05   | 0.7817     | 0.7153     | 0.8617     | 0.985692   | 0.2283     | 68.8s


Default metric period is 5 because AUC is/are not implemented for GPU


8      | 0.07   | 0.7774     | 0.7026     | 0.8700     | 0.992743   | 0.2123     | 65.3s
-----------------------------------------------------------------------------------------------
🏆 Best Tier 2 Model Found!
Params      : Depth 6, LR 0.07
Max F1      : 0.7911
Recall      : 0.7186
Precision   : 0.8798
Rec@0.9     : 0.2380
💾 모델 저장 완료: best_tier2_depth6_lr0.07.cbm


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_curve, confusion_matrix

# 현재 메모리에 있는 최고의 모델 사용
# (만약 날아갔다면: model = CatBoostClassifier(); model.load_model('best_tier2_depth6_lr0.07.cbm'))
final_model = best_patrol['model']

print("🚀 Tier 2: Recall 90% 타겟 설정 중...")

# 1. 예측 확률 다시 계산
y_prob = final_model.predict_proba(X_test)[:, 1]

# 2. PR Curve에서 Recall >= 0.90 인 지점 정밀 탐색
precisions, recalls, thresholds = precision_recall_curve(y_test, y_prob)

# Recall이 0.9 이상인 인덱스들
idx_90 = np.where(recalls[:-1] >= 0.90)[0]

if len(idx_90) > 0:
    # 그 중에서 Precision이 가장 높은 지점 (최적점)
    best_idx = idx_90[np.argmax(precisions[idx_90])]
    
    target_th = thresholds[best_idx]
    target_recall = recalls[best_idx]
    target_prec = precisions[best_idx]
    
    print(f"\n🎯 [Target Found] Recall 90% Cut-off")
    print(f"   - Threshold : {target_th:.8f}  <-- (이 값을 worker.py에 적용하세요!)")
    print(f"   - Recall    : {target_recall:.4f} (목표 달성)")
    print(f"   - Precision : {target_prec:.4f}")

    # ---------------------------------------------------------
    # 3. 최종 시뮬레이션 (Tier 1 + Tier 2 합산)
    # ---------------------------------------------------------
    print("\n" + "="*60)
    print("📊 [최종 확정] FraudGuard Operation Report")
    print("="*60)
    
    # Tier 1 설정 (이미 확정된 값)
    th_tier1 = 0.99816559
    
    # Tier 2 설정 (방금 찾은 값)
    th_tier2 = target_th
    
    n_total = len(y_test)
    n_fraud = sum(y_test)
    
    # Tier 1 (Block)
    mask_t1 = (y_prob >= th_tier1)
    n_t1_fraud = sum(mask_t1 & (y_test == 1))
    n_t1_innocent = sum(mask_t1 & (y_test == 0))
    
    # Tier 2 (Auth) - Tier 1 제외
    mask_t2 = (y_prob >= th_tier2) & (y_prob < th_tier1)
    n_t2_fraud = sum(mask_t2 & (y_test == 1))
    n_t2_innocent = sum(mask_t2 & (y_test == 0))
    
    # Passed
    mask_pass = (y_prob < th_tier2)
    n_pass_fraud = sum(mask_pass & (y_test == 1))
    
    print(f"전체 거래: {n_total:,}건 (사기: {n_fraud:,}건)")
    print("-" * 60)
    
    print(f"🔴 Tier 1 (즉시 차단) : Threshold {th_tier1:.6f}")
    print(f"   - 검거: {n_t1_fraud:,}명 ({(n_t1_fraud/n_fraud*100):.1f}%)")
    print(f"   - 오탐: {n_t1_innocent:,}명 (극소수)")
    print("-" * 60)
    
    print(f"🟡 Tier 2 (추가 인증) : Threshold {th_tier2:.6f}")
    print(f"   - 추가 검거: {n_t2_fraud:,}명 ({(n_t2_fraud/n_fraud*100):.1f}%)")
    print(f"   - 인증 발송: {n_t2_innocent:,}건")
    print(f"   - 발송 비율: 전체 거래의 {(n_t2_innocent/n_total*100):.2f}% (100만 건당 약 {int(n_t2_innocent/n_total*1000000)}건)")
    print("-" * 60)
    
    total_caught = n_t1_fraud + n_t2_fraud
    print(f"🏆 최종 성적 (Tier 1+2)")
    print(f"   - 총 검거율 : {(total_caught/n_fraud*100):.1f}% (목표: 90% 이상)")
    print(f"   - 놓친 사기 : {n_pass_fraud:,}명")
    print("="*60)
    
else:
    print("⚠️ 현재 모델로는 Recall 90% 도달이 불가능합니다. (가중치를 더 줬어야 했을 수도 있습니다)")

🚀 Tier 2: Recall 90% 타겟 설정 중...

🎯 [Target Found] Recall 90% Cut-off
   - Threshold : 0.56802705  <-- (이 값을 worker.py에 적용하세요!)
   - Recall    : 0.9000 (목표 달성)
   - Precision : 0.2380

📊 [최종 확정] FraudGuard Operation Report
전체 거래: 1,830,308건 (사기: 2,761건)
------------------------------------------------------------
🔴 Tier 1 (즉시 차단) : Threshold 0.998166
   - 검거: 1,369명 (49.6%)
   - 오탐: 20명 (극소수)
------------------------------------------------------------
🟡 Tier 2 (추가 인증) : Threshold 0.568027
   - 추가 검거: 1,116명 (40.4%)
   - 인증 발송: 7,934건
   - 발송 비율: 전체 거래의 0.43% (100만 건당 약 4334건)
------------------------------------------------------------
🏆 최종 성적 (Tier 1+2)
   - 총 검거율 : 90.0% (목표: 90% 이상)
   - 놓친 사기 : 276명
